In [1]:
!pip --disable-pip-version-check install sparklines > /dev/null

In [2]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os, sys
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
import re
import glob
import dask.dataframe as dd
from sklearn.preprocessing import MinMaxScaler
import datetime
from pandas.tseries.offsets import MonthBegin
from operator import attrgetter
import sparklines
import base64
from itertools import combinations
from io import BytesIO
from IPython.display import HTML
from plotly.subplots import make_subplots
from warnings import filterwarnings

pd.set_option('MAX_ROWS', None)
pd.set_option('MAX_COLUMNS', None)
pd.set_option('display.max_colwidth', None)
pio.templates.default = "none"
filterwarnings('ignore')

## Problem Statement

The COVID-19 Pandemic has disrupted learning for more than 56 million students in the United States. In the Spring of 2020, most states and local governments across the U.S. closed educational institutions to stop the spread of the virus. In response, schools and teachers have attempted to reach students remotely through distance learning tools and digital platforms. Until today, concerns of the exacaberting digital divide and long-term learning loss among America’s most vulnerable learners continue to grow.

In [3]:
# path of the data
path = r'/kaggle/input/learnplatform-covid19-impact-on-digital-learning/'

## Districts

In [4]:
# read the data
districts = pd.read_csv(os.path.join(path, 'districts_info.csv'))

In [5]:
# shape
districts.shape

(233, 7)

In [6]:
# data types
districts.dtypes

district_id                  int64
state                       object
locale                      object
pct_black/hispanic          object
pct_free/reduced            object
county_connections_ratio    object
pp_total_raw                object
dtype: object

In [7]:
# missing values
districts.isnull().mean()

district_id                 0.000000
state                       0.244635
locale                      0.244635
pct_black/hispanic          0.244635
pct_free/reduced            0.364807
county_connections_ratio    0.304721
pp_total_raw                0.493562
dtype: float64

In [8]:
# duplicates?
districts.duplicated().sum()

0

In [9]:
# change the data type
districts['district_id'] = districts['district_id'].astype(str)

In [10]:
# count of districts by State
districts.groupby('state')['district_id'].size().sort_values(ascending=False)

state
Connecticut             30
Utah                    29
Massachusetts           21
Illinois                18
California              12
Ohio                    11
New York                 8
Indiana                  7
Washington               6
Missouri                 6
Virginia                 4
North Carolina           4
Wisconsin                3
District Of Columbia     3
Texas                    2
Tennessee                2
New Hampshire            2
New Jersey               2
Michigan                 2
North Dakota             1
Minnesota                1
Florida                  1
Arizona                  1
Name: district_id, dtype: int64

In [11]:
# count of locale
districts['locale'].value_counts(dropna=False)

Suburb    104
NaN        57
Rural      33
City       29
Town       10
Name: locale, dtype: int64

In [12]:
# count of percent black/hispanic students
districts['pct_black/hispanic'].value_counts(dropna=False)

[0, 0.2[      116
NaN            57
[0.2, 0.4[     24
[0.4, 0.6[     17
[0.6, 0.8[     11
[0.8, 1[        8
Name: pct_black/hispanic, dtype: int64

In [13]:
# count of percent free/reduced meals
districts['pct_free/reduced'].value_counts(dropna=False)

NaN           85
[0.2, 0.4[    48
[0, 0.2[      46
[0.4, 0.6[    37
[0.6, 0.8[    13
[0.8, 1[       4
Name: pct_free/reduced, dtype: int64

In [14]:
# count of county connections ratio
districts['county_connections_ratio'].value_counts(dropna=False)

[0.18, 1[    161
NaN           71
[1, 2[         1
Name: county_connections_ratio, dtype: int64

In [15]:
# count of per person total expenditure
districts['pp_total_raw'].value_counts(dropna=False)

NaN               115
[8000, 10000[      30
[10000, 12000[     17
[14000, 16000[     15
[12000, 14000[     15
[6000, 8000[       13
[16000, 18000[     13
[18000, 20000[      8
[20000, 22000[      2
[22000, 24000[      2
[4000, 6000[        2
[32000, 34000[      1
Name: pp_total_raw, dtype: int64

In [16]:
# number of districts
districts['district_id'].nunique()

233

### Preprocessing of Districts

Create new features using the ranged field

In [17]:
def split_range(row):
    """
    Split the range related features
    in the Districts dataset, and compute
    the midpoint
    """
    if pd.isna(row):
        return row
    matched = re.search(r'\[(.*?),\s?(.*?)\[', row)
    lb = float(matched.group(1))
    ub = float(matched.group(2))
    return (lb + ub)/2

In [18]:
# create new features
districts['pct_mean_black_hispanic'] = districts['pct_black/hispanic'].apply(split_range)
districts['pct_mean_free'] = districts['pct_free/reduced'].apply(split_range)
districts['county_connections_mean_ratio'] = districts['county_connections_ratio'].apply(split_range)
districts['median_pp_total_raw'] = districts['pp_total_raw'].apply(split_range)

In [19]:
# apply min max scaling due to the wide ranges
minmax = MinMaxScaler()
districts['scaled_black'] = minmax.fit_transform(districts['pct_mean_black_hispanic'].values.reshape(-1, 1))
districts['scaled_free'] = minmax.fit_transform(districts['pct_mean_free'].values.reshape(-1, 1))
districts['scaled_internet'] = minmax.fit_transform(districts['county_connections_mean_ratio'].values.reshape(-1, 1))
districts['scaled_investment'] = minmax.fit_transform(districts['median_pp_total_raw'].values.reshape(-1, 1))

In [20]:
# check top 5 records
districts.head()

,district_id,state,locale,pct_black/hispanic,pct_free/reduced,county_connections_ratio,pp_total_raw,pct_mean_black_hispanic,pct_mean_free,county_connections_mean_ratio,median_pp_total_raw,scaled_black,scaled_free,scaled_internet,scaled_investment
0,8815,Illinois,Suburb,"[0, 0.2[","[0, 0.2[","[0.18, 1[","[14000, 16000[",0.1,0.1,0.59,15000.0,0.0,0.00,0.0,0.357143
1,2685,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4921,Utah,Suburb,"[0, 0.2[","[0.2, 0.4[","[0.18, 1[","[6000, 8000[",0.1,0.3,0.59,7000.0,0.0,0.25,0.0,0.071429
3,3188,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2238,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
# 5 point summary
districts.describe()

,pct_mean_black_hispanic,pct_mean_free,county_connections_mean_ratio,median_pp_total_raw,scaled_black,scaled_free,scaled_internet,scaled_investment
count,176.000000,148.000000,162.000000,118.000000,176.000000,148.000000,162.000000,118.000000
mean,0.239773,0.339189,0.595617,12474.576271,0.174716,0.298986,0.006173,0.266949
std,0.230795,0.212095,0.071496,4524.788071,0.288493,0.265119,0.078567,0.161600
min,0.100000,0.100000,0.590000,5000.000000,0.000000,0.000000,0.000000,0.000000
25%,0.100000,0.100000,0.590000,9000.000000,0.000000,0.000000,0.000000,0.142857
50%,0.100000,0.300000,0.590000,11000.000000,0.000000,0.250000,0.000000,0.214286
75%,0.300000,0.500000,0.590000,15000.000000,0.250000,0.500000,0.000000,0.357143
max,0.900000,0.900000,1.500000,33000.000000,1.000000,1.000000,1.000000,1.000000


## Products

In [22]:
# read the products dataset
products = pd.read_csv(os.path.join(path, 'products_info.csv'))

In [23]:
# top 5 products
products.head()

,LP ID,URL,Product Name,Provider/Company Name,Sector(s),Primary Essential Function
0,13117,https://www.splashmath.com,SplashLearn,StudyPad Inc.,PreK-12,LC - Digital Learning Platforms
1,66933,https://abcmouse.com,ABCmouse.com,"Age of Learning, Inc",PreK-12,LC - Digital Learning Platforms
2,50479,https://www.abcya.com,ABCya!,"ABCya.com, LLC",PreK-12,"LC - Sites, Resources & Reference - Games & Simulations"
3,92993,http://www.aleks.com/,ALEKS,McGraw-Hill PreK-12,PreK-12; Higher Ed,LC - Digital Learning Platforms
4,73104,https://www.achieve3000.com/,Achieve3000,Achieve3000,PreK-12,LC - Digital Learning Platforms


In [24]:
# number of records
products.shape

(372, 6)

In [25]:
# missing values
products.isnull().mean()

LP ID                         0.000000
URL                           0.000000
Product Name                  0.000000
Provider/Company Name         0.002688
Sector(s)                     0.053763
Primary Essential Function    0.053763
dtype: float64

In [26]:
# data types
products.dtypes

LP ID                          int64
URL                           object
Product Name                  object
Provider/Company Name         object
Sector(s)                     object
Primary Essential Function    object
dtype: object

In [27]:
# duplicates?
products.duplicated().sum()

0

In [28]:
# number of unique URLs
products['URL'].nunique()

372

In [29]:
# number of unique Providers
products['Provider/Company Name'].nunique()

290

In [30]:
# count of products for top 5 Providers
products['Provider/Company Name'].value_counts()[:5]

Google LLC                   30
Houghton Mifflin Harcourt     6
Microsoft                     6
IXL Learning                  4
Learning A-Z                  4
Name: Provider/Company Name, dtype: int64

In [31]:
# number of unique sectors
products['Sector(s)'].nunique()

5

In [32]:
# count of top 5 sectors
products['Sector(s)'].value_counts()

PreK-12                          170
PreK-12; Higher Ed; Corporate    115
PreK-12; Higher Ed                65
Corporate                          1
Higher Ed; Corporate               1
Name: Sector(s), dtype: int64

In [33]:
# number of Primary Essential functions
products['Primary Essential Function'].nunique()

35

In [34]:
# count of top 5 Primary Essential functions
products['Primary Essential Function'].value_counts()[:5]

LC - Digital Learning Platforms      74
LC - Sites, Resources & Reference    47
LC - Content Creation & Curation     36
LC - Study Tools                     25
LC - Courseware & Textbooks          18
Name: Primary Essential Function, dtype: int64

## Engagements

Parse the engagement data which is captured at the district level
on a daily basis for the last one year.

In [35]:
def create_engagement_dataset(path, file='*.csv'):
    """
    Create engagement dataset
    """
    all_df = []
    for f in glob.glob(os.path.join(path, file)):
        df = pd.read_csv(f, parse_dates=['time'])
        df['district_id'] = f.split('/')[-1].split('.')[0]
        # df['log_engagement_index'] = np.log1p(df['engagement_index'])
        # df['scaled_access'] = minmax.fit_transform(df['pct_access'].values.reshape(-1, 1))
        # df['scaled_engagement'] = minmax.fit_transform(df['engagement_index'].values.reshape(-1, 1))
        df['usage_month'] = df['time'].dt.to_period('M')
        df['is_pandemic'] = np.where(df['time'] <= datetime.datetime.strptime('2020-07-31', '%Y-%m-%d'), 0, 1)
        all_df.append(df)
    return pd.concat(all_df, ignore_index=True)

In [36]:
%%time
eng_path = r'engagement_data'
daily_eng_df = create_engagement_dataset(os.path.join(path, eng_path))

CPU times: user 19.1 s, sys: 2.79 s, total: 21.9 s
Wall time: 27.9 s


In [37]:
# top 5 engagement records
daily_eng_df.head()

,time,lp_id,pct_access,engagement_index,district_id,usage_month,is_pandemic
0,2020-01-01,92844.0,0.01,0.68,6345,2020-01,0
1,2020-01-01,64838.0,0.01,0.68,6345,2020-01,0
2,2020-01-01,94058.0,0.00,NaN,6345,2020-01,0
3,2020-01-01,26488.0,0.03,26.21,6345,2020-01,0
4,2020-01-01,32340.0,0.01,0.11,6345,2020-01,0


In [38]:
# number of records
daily_eng_df.shape

(22324190, 7)

In [39]:
# number of missing
daily_eng_df.isnull().mean()

time                0.000000
lp_id               0.000024
pct_access          0.000602
engagement_index    0.240923
district_id         0.000000
usage_month         0.000000
is_pandemic         0.000000
dtype: float64

In [40]:
# duplicates?
daily_eng_df.duplicated().sum()

0

In [41]:
# 5 point summary
daily_eng_df.describe()

,lp_id,pct_access,engagement_index,is_pandemic
count,2.232365e+07,2.231074e+07,1.694578e+07,2.232419e+07
mean,5.470879e+04,5.042399e-01,1.676063e+02,4.845449e-01
std,2.647069e+04,3.180568e+00,1.682223e+03,4.997611e-01
min,1.000300e+04,0.000000e+00,1.000000e-02,0.000000e+00
25%,3.085100e+04,0.000000e+00,3.700000e-01,0.000000e+00
50%,5.500700e+04,2.000000e-02,1.920000e+00,0.000000e+00
75%,7.766000e+04,9.000000e-02,1.365000e+01,1.000000e+00
max,9.999100e+04,1.000000e+02,2.130455e+05,1.000000e+00


In [42]:
# drop records with no engagement
daily_eng_df.dropna(inplace=True)

In [43]:
# number of records after removing
# null engagements
daily_eng_df.shape

(16945301, 7)

### Preprocessing of  engagements

1) Merge daily engagements with districts and products.

In [44]:
# merge daily engagement data and products
daily_eng_df = daily_eng_df.merge(products, 
            left_on=['lp_id'], right_on=['LP ID'], how='left')

In [45]:
# unique product ids in products but not in engagement data
len(set(products['LP ID']).difference(set(daily_eng_df['lp_id'])))

3

In [46]:
# unique product ids in the daily engagement data but not in products
len(set(daily_eng_df['lp_id']).difference(set(products['LP ID'])))

7923

In [47]:
# top 5 engagement and products
daily_eng_df.head()

,time,lp_id,pct_access,engagement_index,district_id,usage_month,is_pandemic,LP ID,URL,Product Name,Provider/Company Name,Sector(s),Primary Essential Function
0,2020-01-01,92844.0,0.01,0.68,6345,2020-01,0,92844.0,https://www.typing.com/,Typing.com,Teaching.com,PreK-12; Higher Ed; Corporate,LC - Digital Learning Platforms
1,2020-01-01,64838.0,0.01,0.68,6345,2020-01,0,64838.0,http://www.starfall.com/,Starfall,Starfall Education,PreK-12,"LC - Sites, Resources & Reference - Games & Simulations"
2,2020-01-01,26488.0,0.03,26.21,6345,2020-01,0,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-01-01,32340.0,0.01,0.11,6345,2020-01,0,32340.0,http://genius.com/static/education,Education Genius,Genius Media Group,NaN,NaN
4,2020-01-01,95731.0,0.20,40.96,6345,2020-01,0,95731.0,http://docs.google.com/,Google Docs,Google LLC,PreK-12; Higher Ed; Corporate,LC - Content Creation & Curation


In [48]:
daily_eng_df['scaled_access'] = minmax.fit_transform(daily_eng_df['pct_access'].values.reshape(-1, 1))
daily_eng_df['scaled_engagement'] = minmax.fit_transform(daily_eng_df['engagement_index'].values.reshape(-1, 1))

In [49]:
daily_eng_df = daily_eng_df.merge(districts[['district_id', 'state', 'locale']], 
                                 left_on=['district_id'], right_on=['district_id'], how='left')

In [50]:
# %%time
# # merge daily engagement data and districts
# dist_cols = ['district_id', 'state', 'locale', 'pct_mean_black_hispanic', 'pct_mean_free', 'county_connections_mean_ratio', 'median_pp_total_raw',
#                                 'scaled_black', 'scaled_internet', 'scaled_investment', 'scaled_free']
# daily_eng_df = daily_eng_df.merge(districts[dist_cols], 
#             left_on=['district_id'], right_on=['district_id'], how='left')

In [51]:
# unique district ids in districts  but not in engagement data
# len(set(districts['district_id']).difference(set(daily_eng_df['district_id'])))

In [52]:
# unique district ids in the daily engagement data but not in districts dataframe
# len(set(daily_eng_df['district_id']).difference(set(districts['district_id'])))

In [53]:
# top 5 engagement, products and districts
daily_eng_df.head()

,time,lp_id,pct_access,engagement_index,district_id,usage_month,is_pandemic,LP ID,URL,Product Name,Provider/Company Name,Sector(s),Primary Essential Function,scaled_access,scaled_engagement,state,locale
0,2020-01-01,92844.0,0.01,0.68,6345,2020-01,0,92844.0,https://www.typing.com/,Typing.com,Teaching.com,PreK-12; Higher Ed; Corporate,LC - Digital Learning Platforms,0.0001,3.144869e-06,Massachusetts,Suburb
1,2020-01-01,64838.0,0.01,0.68,6345,2020-01,0,64838.0,http://www.starfall.com/,Starfall,Starfall Education,PreK-12,"LC - Sites, Resources & Reference - Games & Simulations",0.0001,3.144869e-06,Massachusetts,Suburb
2,2020-01-01,26488.0,0.03,26.21,6345,2020-01,0,NaN,NaN,NaN,NaN,NaN,NaN,0.0003,1.229784e-04,Massachusetts,Suburb
3,2020-01-01,32340.0,0.01,0.11,6345,2020-01,0,32340.0,http://genius.com/static/education,Education Genius,Genius Media Group,NaN,NaN,0.0001,4.693834e-07,Massachusetts,Suburb
4,2020-01-01,95731.0,0.20,40.96,6345,2020-01,0,95731.0,http://docs.google.com/,Google Docs,Google LLC,PreK-12; Higher Ed; Corporate,LC - Content Creation & Curation,0.0020,1.922125e-04,Massachusetts,Suburb


### Summary of the 3 datasets

**Districts**


1) There are 233 districts across 23 States in the Districts dataset.

2) Connecticut has the maximum number of districts - 30. There is a non uniform distribution of districts across States.

3) Except for the `distirct_id`, there is data missing across the other features, which ranges from 24% (`state`, `locale`) to 49% (`pp_total_raw`). 

4) The other features describe the percentage of black/hispanic, free/reduced meals, `county_connections_ratio` and total expenditure per person at
a district level. These values are provided as ranges.

5) After taking the midpoint from the ranged values, we can see that the `pct_mean_black_hispanic` and `pct_mean_free` fields range from 10% - 90%.

6) The Internet connection represented by `county_connections_mean_ratio` doesn't show much variation; it is mostly at a ratio of 0.59, which an exception at 1.5.

7) Investment - `median_pp_total_raw` ranges from \\$5000 - \\$33,000, with a mean of ~\\$12,400.

**Products**

1) There are 372 products in use.

2) There are 290 `Provider/Company Name` offering their services across 5 `Sectors`.

3) There are 35 `Primary Essential Function` provided by these companies; the most common
function is LC - Digital Learning Platforms.

**Engagement**

1) There are approximately ~ 22 million engagements for the entire period from 2020-01-01 to 2020-12-31.

2) 24% of the `engagement_index` is missing, and a very small fraction of `lp_id` product_ids are missing.

3) The `pct_access` ranges from 0% - 100%, and the `engagement_index` varies from 100 - 213K.

### Data preparation

1) 24% of the missing `engagement_index` records were removed before merging with Products.

2) The Engagements dataset was merged with Products on `lp_id` and `LP ID`, which resulted in 8818 products ids not being mapped to their
corresponding lookup in the products dataframe. In the Engagements dataset, 7923 product ids are not mapped.

3) Only the State column from Districts dataset was merged to the Engagements and Products to avoid unnecessary increase
in memory usage.


## Deep dive

The analysis takes a look at the various dimensions and their interplay with respect to district dynamics and product characteristics - during Jan2020 - Dec2020,
to understand which districts weathered the storm relatively better, and which ones were severely crippled for one or the other reasons.

In [54]:
def create_table_bar_chart(df, var, bar_color=None, title=None):
    """
    Create a tabular bar chart
    """
    HTML(
        pd.DataFrame(df[var]\
            .value_counts(normalize=True))\
            .style\
            .format('{:.0%}')\
            .set_table_styles([{
            'selector': 'caption',
            'props': [
                ('font-size', '16px')
            ]
            }])\
          .set_caption(title)\
          .set_properties(padding='10px', border='2px solid white')\
          .bar(color=bar_color)
    )

In [55]:
# create_table_bar_chart(daily_eng_df, 'state', bar_color=bar_color, title='Percentage of recorded engagements by State')
bar_color = '#FF7F7F'
pd.DataFrame(daily_eng_df['state'].value_counts(normalize=True)).style\
                                                              .format('{:.0%}')\
                                                              .set_table_styles([{
                                                                'selector': 'caption',
                                                                'props': [
                                                                    ('font-size', '16px')
                                                                ]
                                                            }])\
                                                             .set_caption('Share of daily engagements by State')\
                                                             .set_properties(padding='10px', border='2px solid white')\
                                                             .bar(color=bar_color)


,state
Utah,15%
Connecticut,15%
Illinois,14%
Massachusetts,11%
Ohio,8%
California,7%
Missouri,7%
Washington,5%
Indiana,4%
Virginia,4%


**What do we observe**

1) Utah, Connecticut and Illinois are the top 3 states that showed a relatively better daily engagement share compared to the rest of the States.

2) Some of the poor performing States are: New Hampshire, Arizona, Minnesota and North Dakota.

In [56]:
# top 5 districts in terms of number of engagements recorded
pd.DataFrame(daily_eng_df.groupby(['state','district_id']).size())\
            .sort_values(0, ascending=False)[:5]\
            .rename(columns={0: 'count'})\
            .style\
             .set_table_styles([{
                                'selector': 'caption',
                                'props': [
                                    ('font-size', '16px')
                                ]
                            }])\
            .set_caption('Top 5 districts - Number of daily engagements')\
            .set_properties(padding='10px', border='2px solid white')\
            .bar(color=bar_color)

In [57]:
# bottom 5 districts in terms of number of engagements recorded
pd.DataFrame(daily_eng_df.groupby(['state','district_id']).size())\
            .sort_values(0, ascending=False)[-5:]\
            .rename(columns={0: 'count'})\
            .style\
             .set_table_styles([{
                                'selector': 'caption',
                                'props': [
                                    ('font-size', '16px')
                                ]
                            }])\
            .set_caption('Bottom 5 districts - Number of daily engagements')\
            .set_properties(padding='10px', border='2px solid white')\
            .bar(color=bar_color)

,,count
state,district_id,
Tennessee,8103,9967
Illinois,5042,8067
Utah,4373,7951
North Dakota,2872,1607
Ohio,3670,1401


**What do we observe**

1) There is a lot of variation within a State itself. For example, district - 8784 in Illinois has the highest registered number of daily engagements ~233K. However, in contrast, another district 5042 in the same state
has one of the lowest number of engagements - ~5K.

<a id=mean_monthly_engagement_index></a>

In [58]:
# time-series plot of mean monthly engagement_index 
overall_mean_eng_df = daily_eng_df[['time', 'pct_access', 'engagement_index']].copy()
overall_mean_eng_df.set_index('time', inplace=True)
overall_mean_eng_df = overall_mean_eng_df.resample('1M').mean()
overall_mean_eng_df.index = overall_mean_eng_df.index - MonthBegin(1)
fig = px.line(overall_mean_eng_df, y='engagement_index',
              title='Mean monthly engagement index across all districts')
fig.update_xaxes(dtick="M1",
                 tickformat="%b\n%Y")


**What do we observe**

`engagement_index` - the total page load events per 1000 students for a given product on a given day

1) Schools were given orders to shut down in March. However, from this chart, it took a while before engagement waned. The month
of July witnessed the lowest `engagement_index` of 50 page load events per 1000 students. However, we see a reversal in this trend, which peaks in Septemeber, and dipped slightly
towards the final quarter in that year.

In [59]:
# time-series plot of mean monthly percent access
fig = px.line(overall_mean_eng_df, y='pct_access',
              title='Mean monthly percent of access across all districts')
fig.update_xaxes(dtick="M1",
                 tickformat="%b\n%Y")


**What do we observe**

`pct_access` - Percentage of students in the district have at least one page-load event of a given product and on a given day

1) The `pct_access` is at its highest in January. From here onwards, it drops gradually witnessing a steep decline starting from May onwards hitting its nadir in July. Just like
with `engagement_index`, there is a reversal in this trend peaking in Septemeber, and dipping towards the last quarter of 2020.

In [60]:
def create_data_for_various_plots(df, field, freq='1D', 
                                eng_cols=None,
                                agg_var = None, 
                                is_state_level=False,
                                is_district_level=False,
                                other_df=None,
                                ):
    """
    Create data to plot scatter plot
    This is to show daily/monthly engagement
    """
    if is_state_level:
        df = df[eng_cols].groupby(['state']).agg(agg_var).reset_index()
        return df.merge(other_df, left_on=['state'],
                     right_on=['state'], how='left')
    else:
        df = df[eng_cols].groupby(['state', 'district_id']).agg(agg_var).reset_index()
        return df.merge(other_df, left_on=['state', 'district_id'],
                     right_on=['state', 'district_id'], how='left')
#     else:
#         # daily at the time level
#         if not is_state_level:
#             return df.groupby(pd.Grouper(key=field, freq=freq)).mean()
#         else:
#             # daily at the state level
#             df = df[eng_cols].groupby(['time', 'state']).agg(agg_var).reset_index()

## State-wise characteristics

Analyse the state and district-wise characteristics

In [61]:
# aggregate the district-wise characteristtics at the State level
states_agg = districts.groupby(['state']).agg({'pct_mean_black_hispanic': np.mean,
                                               'pct_mean_free': np.mean,
                                    'county_connections_mean_ratio': np.mean,
                                    'median_pp_total_raw': np.mean,
                                    'scaled_black': np.mean,
                                    'scaled_free': np.mean,
                                    'scaled_internet': np.mean,
                                    'scaled_investment': np.mean}).reset_index()

In [62]:
%%time
# prepare state level engagement data for analysis of state-wise characteristics
dist_cols = ['state', 'pct_mean_black_hispanic', 'pct_mean_free', 'county_connections_mean_ratio', 'median_pp_total_raw',
            'scaled_black', 'scaled_free', 'scaled_investment']
eng_cols = ['time', 'state', 'pct_access', 'engagement_index', 'scaled_access', 'scaled_engagement']
state_level_data_df = create_data_for_various_plots(daily_eng_df, 'time',
                                                      eng_cols = eng_cols,
                                                      agg_var = {'pct_access': np.mean,
                                                                'engagement_index': np.mean,
                                                                'scaled_access': np.mean,
                                                                'scaled_engagement': np.mean},
                                                      is_state_level=True,
                                                      other_df=states_agg[dist_cols])

CPU times: user 2.35 s, sys: 247 ms, total: 2.6 s
Wall time: 2.6 s


In [63]:
%%time
# prepare district level data for analysis of district-wise characteristics
district_level_data_df = create_data_for_various_plots(daily_eng_df, 'time',
                                                      eng_cols = eng_cols + ['district_id'],
                                                      agg_var = {'pct_access': np.mean,
                                                                'engagement_index': np.mean,
                                                                'scaled_access': np.mean,
                                                                'scaled_engagement': np.mean},
                                                      is_district_level=True,
                                                      other_df=districts[dist_cols + ['district_id']],
                                                      )

CPU times: user 4.06 s, sys: 750 ms, total: 4.81 s
Wall time: 4.81 s


In [64]:
# aggregate the district features by state and melt the dataframe
# monthly_state_data_df = state_level_data_df.groupby('state').mean().reset_index()
state_level_data_melted_df  = pd.melt(state_level_data_df, id_vars=['state'], value_vars=['scaled_access', 'scaled_engagement', 
                                                             'scaled_black', 'scaled_free', 
                                                             'scaled_investment'], 
            value_name='feature_value', var_name='feature_parameter')
# sort the dataframe in descending order of mean value for each feature_parameter
state_level_data_melted_df = state_level_data_melted_df.groupby(['feature_parameter'])['state', \
                                                           'feature_value']\
                                                        .apply(lambda x: x.sort_values('feature_value', ascending=False))\
                                                        .reset_index()\
                                                        .drop(['level_1'], axis=1)
cols = ['scaled_access', 'scaled_engagement', 'scaled_black',
       'scaled_free', 'scaled_investment']
# add column to capture the mean value for each parameter
for col in cols:
    mean_col_value = state_level_data_melted_df.query(f'feature_parameter == "{col}"')['feature_value'].mean()
    
    subset_df = state_level_data_melted_df.query(f'feature_parameter == "{col}"')
    idx = subset_df['feature_value'].apply(lambda x: 'Above mean' if x > mean_col_value else 'Below mean').index
    state_level_data_melted_df.loc[idx, 'color'] = subset_df['feature_value'].apply(lambda x: 'Above mean' if x > mean_col_value else 'Below mean')
# rename the feature_parameter
map_labels = {
    'scaled_access': 'Percent access of at least one page load event',
    'scaled_engagement': 'Page load events per 1000 students',
    'scaled_black': 'Percent of reported Hispanic/Black students',
    'scaled_free': 'Percent of free/reduced price meal',
    'scaled_investment': 'Median per pupil expenditure'
}
state_level_data_melted_df['new_feature_parameter'] = state_level_data_melted_df['feature_parameter'].replace(map_labels)

In [65]:
def make_subplots_for_bar_charts(df, series,
                  colors=['rgb(255, 0, 0)', '#2ca02c'],
                  title=None
                 ):
    """
    Helper function to plot scatter charts using subplots
    """
    fig = px.bar(df, x=series, y='feature_value', facet_col='new_feature_parameter', facet_col_wrap=2, 
                facet_row_spacing=0.3, facet_col_spacing=0.1, height=1200, width=900, color='color', 
                 color_discrete_map={'Above mean': colors[1],
                                    'Below mean': colors[0]})
    fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
    fig.layout.xaxis2.update(matches=None)
    fig.update_xaxes(matches=None, showticklabels=True)
    fig.update_yaxes(matches=None, showticklabels=True)
    fig.update_yaxes(visible=False, showticklabels=False)
    fig.update_layout(showlegend=False, title=title)
    for axis in fig.layout:
        if type(fig.layout[axis]) == go.layout.YAxis:
            fig.layout[axis].title.text = ''
        if type(fig.layout[axis]) == go.layout.XAxis:
            fig.layout[axis].title.text = ''
    fig.show()

In [66]:
make_subplots_for_bar_charts(state_level_data_melted_df, 'state',
                             title='Bar charts displaying district features by State'
                 )

**How is the data prepared**

1) Daily engagements are aggregated at the State level and the mean `engagement_index` for each State is computed; this is joined with the district dataframe to capture the district features.

2) The conditional formatting is applied using the mean `engagement_index` - if it is above the mean, green is used to represent the State, otherwise red is used.

**What do we observe**

1) Utah may have the highest share of daily engagements, however, with respect to the `pct_access` and `engagement_index` features, they are below the mean.

2) States such as North Dakota has a higher than average `pct_access`, but the `engagement_index` is below the overall States mean. This is because it has data for the first 3 months only.

3) Arizona has the highest mean `engagement_index` and `pct_access` of all States; it also has only one district captured in this dataset.

4) Minnesota, Indiana, and Michigan, in that order, have the highest percentage of free/reduced meal price of all States.

5) New York, District Of Columbia, and New Jersey, in that order have the highest median per pupil expenditure of all States.

6) Quite a number of states haven't reported figures for percentage of reported Black/Hispanic students, percent of free/reduced mean price and median per pupil expenditure.

### Correlation of district features

<a id=correlation_matrix></a>

In [67]:
cols_compare = ['pct_access', 'engagement_index', 'pct_mean_black_hispanic',
               'pct_mean_free', 'median_pp_total_raw', 'county_connections_mean_ratio']
dist_corr = district_level_data_df[cols_compare].corr()
N = len(dist_corr.columns)
X = dist_corr.columns.tolist()
mask = np.zeros_like(dist_corr, dtype = np.bool)
mask[np.triu_indices_from(mask)] = True
dist_corr=dist_corr.mask(mask)
hovertext = [[f'corr({X[i]}, {X[j]}) = {dist_corr.values[i][j]:.0%}' if i > j else '' for j in range(N)] for i in range(N)]
heat = go.Heatmap(z=dist_corr,
                  x=X,
                  y=X,
                  xgap=1, ygap=1,
                  colorscale='rdylgn',
                  colorbar_thickness=20,
                  colorbar_ticklen=3,
                  hovertext=hovertext,
                  hoverinfo='text'
                   )


title = 'Correlation plot for district level features'               

layout = go.Layout(title_text=title, title_x=0.5, 
                   width=800, height=800,
                   xaxis_showgrid=False,
                   yaxis_showgrid=False,
                   yaxis_automargin=True,
                   xaxis_automargin=True,
                   yaxis_autorange='reversed')
   
fig=go.Figure(data=[heat], layout=layout)        
fig.show() 

**How is the data prepared**

1) Daily engagements are aggregated at the State and district level and, the mean `engagement_index` for each State and district is computed; this is joined with the district dataframe to capture the district features.

**What do we observe**

1) Not all district features are strong correlated.

2) The only relationships that stand out are: `pct_access` versus `engagement_index`. b) `engagement_index` versus `median_pp_total_raw`. c) `pct_mean_free` versus `pct_mean_black_hispanic`. All of these relationships
are positively correlated.

3) There is a moderately strong positive correlation between `pct_access` and `county_connections_means_ratio`.

## District-wise correlations

In [68]:
def create_parallel_coord(df, states):

    fig = go.Figure(data=
                    go.Parcoords(
                    line = dict(color=df['district_id'],
                              colorscale = [[0,'purple'],[0.5,'lightseagreen'],[1,'gold']]),
                    dimensions = list([
                        dict(
                                label = 'Districts', values = df['district_id']),
                        dict(range = [0, 1],
                                label = 'Black/Hispanic', values = df['scaled_black']),
                        dict(range = [0, 1],
                                label = 'Discounted meal', values = df['scaled_free']),
#                         dict(range = [0, .2],
#                                 label = 'Internet', values = df['scaled_internet']),
                        dict(range = [0, 1],
                                constraintrange = [0, .5], 
                                label = 'Investment', values = df['scaled_investment']),
                        dict(range = [0, .2],
                                label = 'Access', values = df['scaled_access']),
                        dict(range = [0, .007],
                                label = 'Engagement', values = df['scaled_engagement']),

                    ])
        )
    )


    fig.update_layout(title=f'Relationship between the district features in the state of {state}')
    fig.show()

<a id=parallel_coord></a>

In [69]:
state = 'New York'
district_level_data_df['district_id'] = district_level_data_df['district_id'].astype('int32')
create_parallel_coord(district_level_data_df.query(f'state == "{state}"'), state)

**What do we observe**

1) The chart shows how the various district level features interact with one another. All the features, execept for the `distrct_id` were scaled to deal with the outliers.

2) District 9536 stands out here; it has both a high percentage of reported Black/Hispanics students and a high percentage of students eligible for free/reduced price lunch measl. The dsitrict, however, has no reported
per pupil expenditure, but its `pct_access` and `engagement_index` metrics are the best of the lot.


In [70]:
state = 'District Of Columbia'
create_parallel_coord(district_level_data_df.query(f'state == "{state}"'), state)

**What do we observe**

1) We have data for only 3 districts in this State. District 6500 is the only one with the highest `pct_access` and `engagement_index` metrics of all districts in DC.

In [71]:
state = 'Michigan'
create_parallel_coord(district_level_data_df.query(f'state == "{state}"'), state)

**What do we observe**

1) The 2 reported districts appear to have very low `pct_access` and `engagement_index` values during the year.

In [72]:
state = 'Texas'
create_parallel_coord(district_level_data_df.query(f'state == "{state}"'), state)

**What do we observe**

1) The pattern here is similar to what we observe for State of Michigan.

In [73]:
def data_for_sparkline(df, grp_var, time_var, max_time='2020-12',
                              min_time='2020-01',
                              agg_var=None,
                              is_state=False,
                              is_district=False,
                              is_product=False, 
                              is_product_eng=False,
                              is_product_combo=False,
                              is_product_combo_eng=False,
                              is_multi_level=False, 
                              cust_sparkline=True):
    """
    Create data for sparkline
    """
    tmp_df = df.copy()
    if is_state:
        g = tmp_df.groupby(grp_var).agg({agg_var: np.sum,})
        g = g.groupby(level=[0]).apply(lambda x: x/x.sum()).reset_index()
    elif is_district:
        g = tmp_df.groupby(grp_var).agg({agg_var: np.sum,})
        g = g.groupby(level=[0, 1]).apply(lambda x: x/x.sum()).reset_index()
    elif is_product:
        g = tmp_df.groupby(grp_var).size()
        if is_multi_level:
            g = g.groupby(level=[0, 1]).apply(lambda x: x/x.sum()).reset_index().rename(columns={0: 'usage_value'})
        else:
            g = g.groupby(level=[0]).apply(lambda x: x/x.sum()).reset_index().rename(columns={0: 'usage_value'})
    elif is_product_eng:
        g = tmp_df.groupby(grp_var).agg({agg_var: np.sum,})
        if is_multi_level:
            g = g.groupby(level=[0, 1]).apply(lambda x: x/x.sum()).reset_index().rename(columns={0: 'usage_value'})
        else:
            g = g.groupby(level=[0]).apply(lambda x: x/x.sum()).reset_index().rename(columns={0: 'usage_value'})
    elif is_product_combo:
        g = tmp_df.groupby(grp_var).size()
        if is_multi_level:
            g = g.groupby(level=[0, 1, 2]).apply(lambda x: x/x.sum()).reset_index().rename(columns={0: 'usage_value'})
        else:
            g = g.groupby(level=[0, 1]).apply(lambda x: x/x.sum()).reset_index().rename(columns={0: 'usage_value'})
    elif is_product_combo_eng:
        g = tmp_df.groupby(grp_var).agg({agg_var: np.sum,})
        if is_multi_level:
            g = g.groupby(level=[0, 1, 2]).apply(lambda x: x/x.sum()).reset_index().rename(columns={0: 'usage_value'})
        else:
            g = g.groupby(level=[0, 1]).apply(lambda x: x/x.sum()).reset_index().rename(columns={0: 'usage_value'})
    
    # grp_mean = g.iloc[:, -1].mean() # compute mean for the last column
    g['usage_month'] = g['usage_month'].astype(str)
    g = g.pivot_table(index=grp_var[:-1], columns='usage_month', fill_value=0)
    g.columns = g.columns.droplevel() # drop usage_value
    g = g.rename_axis(None, axis=1) # remove usage_month
    if cust_sparkline:
        g['trend'] = g.apply(custom_sparkline, axis=1)
    else:
        g['trend'] = g.apply(lambda x: sparklines.sparklines(x)[0], axis=1)
    g['growth'] = np.round((g[max_time] / g[min_time]) ** (1/12) - 1, 2)
    g['growth'] = g['growth'].replace(np.inf, 0).replace(np.nan, 0)
    return g

In [74]:
def highlight_table(row, threshold=.05):
    """
    Helper function to highlight cells
    in a Pandas dataframe
    """
    if isinstance(row[0], str): return
    return [
        'background-color: #FF7F7F; color: white' if cell <= threshold
        else 'background-color: green; color: white'
        for cell in row
    ]

In [75]:
def custom_sparkline(data, figsize=(3, 0.25), **kwags):
    """
    Create a sparkline chart
    https://github.com/iiSeymour/sparkline-nb/blob/master/sparkline-nb.ipynb
    """
    data = list(data)
    fig, ax = plt.subplots(1, 1, figsize=figsize, **kwags)
    ax.plot(data)
    for k, v in ax.spines.items():
        v.set_visible(False)
    
    ax.set_xticks([])
    ax.set_yticks([])
    
    plt.plot(len(data)-1, data[len(data)-1], 'r.', linewidth=2)
    
    # ax.fill_between(range(len(data)), data, len(data)*[min(data)], alpha=0.1)
    
    img = BytesIO()
    plt.savefig(img, transparent=True, dpi=100, bbox_inches='tight')
    # img.seek(0)
    plt.close()
    
    return f'<img src="data:image/png;base64,{base64.b64encode(img.getvalue()).decode()}"/>'
    # return display(HTML(df.to_html(escape=False))

## Engagement index share by State and District over time

In [76]:
%%time
# subset the daily engagement data to analyze the month on month
# mean engagement at state level
tmp_df = daily_eng_df[['state', 'district_id', 'usage_month', 'scaled_engagement', 'scaled_access']].copy()
state_wise_mom_engagement_growth = data_for_sparkline(tmp_df, ['state', 'usage_month'], 'time', max_time='2020-12',
                              min_time='2020-01',
                              agg_var='scaled_engagement',
                              is_state=True,
                              cust_sparkline=False)

CPU times: user 2.97 s, sys: 1.13 s, total: 4.1 s
Wall time: 4.1 s


<a id=State-wise_monthly_share_of_engagement_index></a>

In [77]:
# mean_eng_threshold = daily_eng_df.groupby(['state'.mean()
grad_cols = state_wise_mom_engagement_growth.columns.drop(['trend', 'growth']).tolist()
state_wise_mom_engagement_growth.sort_values(['growth', 'state'], ascending=[False, True], kind='mergesort').style\
                                            .format('{:.1%}', subset=['growth'])\
                                            .format('{:.1%}', subset=state_wise_mom_engagement_growth.columns.drop(['trend', 'growth']))\
                                            .set_table_styles([{
                                                'selector': 'caption',
                                                'props': [
                                                    ('font-size', '16px')
                                                ]
                                            }])\
                                            .set_caption('Share in engagement index by State')\
                                            .set_properties(padding='10px', border='2px solid white')\
                                            .background_gradient(cmap='RdYlGn', subset=grad_cols, axis=1)\
                                            .background_gradient(cmap='RdYlGn', subset=['growth'], axis=0)
  #                                          .apply(highlight_table, args=(state_eng_mean, ), axis=0)\

,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,2020-11,2020-12,trend,growth
state,,,,,,,,,,,,,,
District Of Columbia,2.7%,3.2%,4.9%,4.6%,4.1%,1.4%,1.1%,4.8%,22.9%,21.0%,15.0%,14.3%,▂▂▂▂▂▁▁▂█▇▅▅,15.0%
New York,2.0%,5.7%,14.8%,14.6%,12.1%,6.6%,1.0%,1.2%,10.4%,12.0%,9.9%,9.7%,▁▃██▇▄▁▁▆▇▆▅,14.0%
Texas,3.5%,6.6%,0.6%,0.0%,7.5%,1.9%,1.5%,7.7%,21.6%,20.7%,14.5%,13.7%,▂▃▁▁▃▂▁▄██▆▅,12.0%
Michigan,4.6%,6.1%,4.1%,1.2%,1.8%,0.3%,0.1%,0.8%,21.7%,23.6%,19.1%,16.4%,▂▃▂▁▂▁▁▁▇█▇▆,11.0%
Illinois,3.6%,6.2%,9.9%,12.2%,7.7%,1.2%,0.8%,6.4%,15.2%,14.2%,11.1%,11.4%,▂▄▅▇▄▁▁▄█▇▆▆,10.0%
California,3.9%,5.4%,7.6%,8.0%,8.0%,1.8%,1.0%,8.8%,17.5%,16.0%,11.2%,10.8%,▂▃▄▄▄▁▁▄█▇▅▅,9.0%
Connecticut,4.2%,5.7%,10.1%,12.5%,11.2%,4.7%,0.8%,0.9%,11.7%,13.9%,12.1%,12.2%,▃▄▆▇▇▃▁▁▇█▇▇,9.0%
Florida,4.4%,7.7%,5.4%,12.9%,6.6%,1.0%,1.3%,7.7%,14.0%,16.8%,11.3%,10.9%,▃▄▃▆▄▁▁▄▇█▆▅,8.0%
Virginia,4.1%,10.8%,9.2%,7.3%,6.5%,1.5%,1.0%,1.3%,16.5%,18.1%,13.2%,10.6%,▂▅▄▄▃▁▁▁▇█▆▅,8.0%


**How is the data prepared**

1) Each row sums to 100%; it represents the share of `engagement_index` for a State during the period from Jan-Dec.

2) The mean monthly `engagement_index` growth is computed using the formula - Growth rate = (Present value / Past value) 1/N - 1 for N=12.

3) The gradient color scheme automatically highlights the cells depending on the size of the value.

**What do we observe**

1) Schools were on the learning platform before the pandemic as evidenced by the engagement numbers in the first quarter. However, their share of `engagement_index`was relatively lower.

2) District Of Columbia (DC) has the highest `engagement_index` growth rate of 15%, closely followed by New York at 14%. It must be noted though that, looking at the humps in the trend graph, most of DCs growth came in the last 4 months.

3) While some States couldn't deal with the stress induced by the pandemic, other States such as New York, Illinois, and Connecticut, to name a few, showed positive signs of engagement, but all States eventually succumbed between the 2nd and 3rd quarter.

4) Most States showed some uptick in engagement rates towards the last quarter as shown in the positive growth rates.

5) States such as Minnesota and North Dakota had among the lowest engagement growth rates.


### Percent access growth rate of at least one page load event - State level

In [78]:
%%time
# subset the daily engagement data to analyze the month on month
# mean engagement
state_wise_mom_pct_access_growth = data_for_sparkline(tmp_df, ['state', 'usage_month'], 'time', max_time='2020-12',
                              min_time='2020-01',
                              agg_var='scaled_access',
                              is_state=True,
                              cust_sparkline=False)

CPU times: user 2.53 s, sys: 567 ms, total: 3.09 s
Wall time: 3.09 s


In [79]:
# mean_access_threshold = daily_eng_df['scaled_access'].mean()
state_wise_mom_pct_access_growth.sort_values(['growth', 'state'], ascending=[False, True], kind='mergesort').style\
                                            .format('{:.1%}', subset=['growth'])\
                                            .format('{:.1%}', subset=state_wise_mom_pct_access_growth.columns.drop(['trend', 'growth']))\
                                            .set_table_styles([{
                                                'selector': 'caption',
                                                'props': [
                                                    ('font-size', '16px')
                                                ]
                                            }])\
                                            .set_caption('Share in percentage of access by State')\
                                            .set_properties(padding='10px', border='2px solid white')\
                                            .background_gradient(cmap='RdYlGn', subset=grad_cols, axis=1)\
                                            .background_gradient(cmap='RdYlGn', subset=['growth'], axis=0)
 #                                           .apply(highlight_table, args=(state_eng_mean, ), axis=0)

,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,2020-11,2020-12,trend,growth
state,,,,,,,,,,,,,,
New York,4.4%,7.5%,12.7%,12.5%,11.3%,6.2%,0.9%,1.0%,10.6%,12.6%,10.9%,9.5%,▃▅██▇▄▁▁▇█▇▆,7.0%
District Of Columbia,6.1%,5.1%,4.6%,4.1%,4.1%,1.6%,1.1%,4.2%,21.1%,20.1%,15.0%,12.8%,▃▂▂▂▂▁▁▂██▆▅,6.0%
Michigan,9.0%,7.7%,4.3%,1.4%,2.1%,0.4%,0.1%,0.8%,20.3%,22.0%,18.1%,13.9%,▄▃▂▁▂▁▁▁▇█▇▅,4.0%
Illinois,7.5%,7.8%,9.1%,10.7%,7.2%,1.2%,0.8%,5.5%,14.2%,14.0%,11.4%,10.4%,▄▅▅▆▄▁▁▃██▇▆,3.0%
Texas,8.3%,9.2%,0.7%,0.0%,7.4%,2.0%,1.5%,6.7%,19.7%,18.7%,13.7%,11.9%,▄▄▁▁▄▂▂▃██▆▅,3.0%
Connecticut,8.6%,7.6%,8.5%,10.2%,10.2%,4.6%,0.9%,0.9%,11.3%,13.8%,12.2%,11.2%,▅▅▅▆▆▃▁▁▇█▇▇,2.0%
North Carolina,12.2%,11.8%,6.9%,0.8%,0.5%,0.2%,0.1%,2.9%,9.7%,21.3%,17.3%,16.3%,▅▅▃▁▁▁▁▂▄█▇▆,2.0%
Tennessee,12.0%,13.0%,2.3%,1.1%,0.3%,0.1%,0.0%,8.0%,20.2%,15.3%,14.0%,13.7%,▅▆▂▁▁▁▁▄█▆▆▆,1.0%
Utah,8.7%,9.3%,11.1%,10.8%,6.9%,0.5%,0.3%,4.3%,13.0%,12.6%,12.8%,9.8%,▆▆▇▇▅▁▁▃███▆,1.0%


**What do we observe**

1) The top 3 States with the highest growth rate of students having accessed at least one page load event, are: New York (7%), District Of Columbia (6%) and Michigan (4%). However, we can see with the State of DC, the `pct_access` was relatively lower in the first half, excluding June and July, compared to its peers.

2) The other States display some healthy access activity during the first quarter before waning during the second and third quarter.

## Engagement index growth comparison - District level

In [80]:
%%time
# subset the daily engagement data to analyze the month on month
# mean engagement at state level
district_wise_mom_engagement_growth = data_for_sparkline(tmp_df, ['state', 'district_id', 'usage_month'], 'time', max_time='2020-12',
                              min_time='2020-01',
                              agg_var='scaled_engagement',
                              is_district=True,
                              cust_sparkline=False)

CPU times: user 4.1 s, sys: 954 ms, total: 5.06 s
Wall time: 5.05 s


In [81]:
# mean_eng_threshold = daily_eng_df['scaled_engagement'].mean()
district_wise_mom_engagement_growth.sort_values(['growth', 'state'], ascending=[False, True], kind='mergesort')[:10].style\
                                            .format('{:.1%}', subset=['growth'])\
                                            .format('{:.1%}', subset=district_wise_mom_engagement_growth.columns.drop(['trend', 'growth']))\
                                            .set_table_styles([{
                                                'selector': 'caption',
                                                'props': [
                                                    ('font-size', '16px')
                                                ]
                                            }])\
                                            .set_caption('Share in engagement index of top 10 Districts')\
                                            .set_properties(padding='10px', border='2px solid white')\
                                            .background_gradient(cmap='RdYlGn', subset=grad_cols, axis=1)\
                                            .background_gradient(cmap='RdYlGn', subset=['growth'], axis=0)
 #                                           .apply(highlight_table, args=(mean_dist_eng, ), axis=0)

**What we observe**

1) For the top 10 districts, the first 2-3 months have seen a slow adoption of digital technology. For districts such as 9536 (IL), (8520, 2257)(NY),
there was some engagement activity, going by the humps observed in the trend graph, before all the districts turned red towards the end of the 2nd quarter and the start of the 3rd quarter. However, they have
all managed to pick up pace in the last quarter.

2) It appears that some districts despite the call for lockdown were able to cope with the shock relatively better than other districts during April and May. However, all of them showed remarkable improvement in the last 4 months.


In [82]:
district_wise_mom_engagement_growth.sort_values(['growth', 'state'], ascending=[False, True], kind='mergesort')[-10:].style\
                                            .format('{:.1%}', subset=['growth'])\
                                            .format('{:.1%}', subset=district_wise_mom_engagement_growth.columns.drop(['trend', 'growth']))\
                                            .set_table_styles([{
                                                'selector': 'caption',
                                                'props': [
                                                    ('font-size', '16px')
                                                ]
                                            }])\
                                            .set_caption('Share in engagement index of bottom 10 Districts')\
                                            .set_properties(padding='10px', border='2px solid white')\
                                            .background_gradient(cmap='RdYlGn', subset=grad_cols, axis=1)\
                                            .background_gradient(cmap='RdYlGn', subset=['growth'], axis=0)
 #                                           .apply(highlight_table, args=(mean_dist_eng, ), axis=0)

,,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,2020-11,2020-12,trend,growth
state,district_id,,,,,,,,,,,,,,
Wisconsin,7752,9.9%,13.5%,13.2%,25.8%,19.9%,3.0%,0.7%,0.1%,14.0%,0.0%,0.0%,0.0%,▄▅▅█▆▂▁▁▅▁▁▁,-66.0%
New York,2940,26.2%,28.6%,45.2%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,▅▅█▁▁▁▁▁▁▁▁▁,-68.0%
Wisconsin,8902,18.5%,29.5%,36.3%,15.7%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,▅▇█▄▁▁▁▁▁▁▁▁,-68.0%
Ohio,6104,13.9%,21.4%,26.5%,27.1%,10.7%,0.3%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,▅▇██▄▁▁▁▁▁▁▁,-72.0%
Massachusetts,7723,12.8%,13.5%,13.5%,19.0%,23.4%,12.7%,3.7%,1.6%,0.0%,0.0%,0.0%,0.0%,▅▅▅▇█▅▂▁▁▁▁▁,-100.0%
New York,4775,8.0%,35.5%,56.4%,0.0%,0.1%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,▂▅█▁▁▁▁▁▁▁▁▁,-100.0%
North Dakota,2872,100.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,█▁▁▁▁▁▁▁▁▁▁▁,-100.0%
Ohio,8425,8.5%,12.9%,19.3%,26.1%,16.4%,1.3%,1.8%,4.1%,9.6%,0.0%,0.0%,0.0%,▃▄▆█▅▁▁▂▄▁▁▁,-100.0%
Tennessee,8103,30.0%,43.3%,26.3%,0.2%,0.1%,0.0%,0.1%,0.0%,0.0%,0.0%,0.0%,0.0%,▆█▅▁▁▁▁▁▁▁▁▁,-100.0%


**What do we observe**

1) In contrast, most of the bottom 10 districts seem to have little to no engagement data starting from end of the first half and continuing on to the second half of the year; this could be attributed due to several reasons:

    a) Challenge with teachers and/or students not attending class due to personal emergencies and the general chaos we were all living in.

    b) Challenges with internet speed and/or the usage of digital learning medium.

    c) Issues with the learning platform not recording data. 

    d) Limited intervention from the School and district operations management team.

## Products characteristics

In [83]:
# filter the null values in the LP ID and state level
lp_daily_eng_df = daily_eng_df[(daily_eng_df['LP ID'].notnull()) & (daily_eng_df['state'].notnull())].reset_index(drop=True)

In [84]:
lp_daily_eng_df.shape

(7313765, 17)

In [85]:
lp_daily_eng_df.isnull().mean()

time                          0.000000
lp_id                         0.000000
pct_access                    0.000000
engagement_index              0.000000
district_id                   0.000000
usage_month                   0.000000
is_pandemic                   0.000000
LP ID                         0.000000
URL                           0.000000
Product Name                  0.000000
Provider/Company Name         0.000000
Sector(s)                     0.040953
Primary Essential Function    0.040953
scaled_access                 0.000000
scaled_engagement             0.000000
state                         0.000000
locale                        0.000000
dtype: float64

In [86]:
monthly_product_usage = lp_daily_eng_df.groupby(['time'])['LP ID'].size()
#monthly_product_usage.set_index('time', inplace=True)
monthly_product_usage = monthly_product_usage.resample('1M').sum() / monthly_product_usage.sum()
monthly_product_usage.index = monthly_product_usage.index - MonthBegin(1)

In [87]:
# plot the monthly share of product usage
fig = px.line(monthly_product_usage, y='LP ID', title='Monthly frequency of product usage', 
       )
max_yaxis = monthly_product_usage.max()
fig.update_yaxes(tickformat=".0%",
                range=[0, np.round(max_yaxis, 2)],
                )
fig.update_xaxes(dtick="M1",
                 tickformat="%b\n%Y")

**How was the data prepared**

1) The records with missing product_id and State were filtered out for this part of the analysis; this resulted in filtering 57% of the data. The loss of data doesn't change the overall distribution in terms of frequency of usage
significantly.

**What do we observe**

1) There was an adoption of digital learning technologies prior to the start of the pandemic, which appears to have peaked at about the time schools were asked to shutdown in late March. As expected, usage dips to its lowest in July, and 
then peaks to about 10% in October, before dipping in the last two months.

## Top 10 products used

In [88]:
# distribution before removal of missing product_id and state
daily_eng_df['Product Name'].value_counts(normalize=True)[:10]

Google Docs         0.007948
Google Drive        0.007687
Google Classroom    0.007396
Google Forms        0.007176
Google Sites        0.007107
Chrome Web Store    0.007100
Wikipedia           0.007087
Google Sheets       0.007034
Khan Academy        0.006775
Prodigy             0.006605
Name: Product Name, dtype: float64

In [89]:
pd.DataFrame(lp_daily_eng_df['Product Name']\
            .value_counts(normalize=True))[:10]\
            .style\
            .format('{:.0%}')\
            .set_table_styles([{
                'selector': 'caption',
                'props': [
                    ('font-size', '16px')
                ]
             }])\
            .set_caption('Share of top 10 daily used products')\
            .set_properties(padding='10px', border='2px solid white')\
            .bar(color=bar_color)

,Product Name
Google Docs,1%
Google Drive,1%
Google Classroom,1%
Google Forms,1%
Google Sites,1%
Google Sheets,1%
Wikipedia,1%
Chrome Web Store,1%
Khan Academy,1%
Quizlet,1%


**What do we observe**

1) Mostly Google related products appear to be frequently used by the students. However its share is very small. Additionally, all the top 10 products
have similar shares.

## Top 10 providers

In [90]:
pd.DataFrame(lp_daily_eng_df['Provider/Company Name']\
            .value_counts(normalize=True))[:10]\
            .style\
            .format('{:.0%}')\
            .set_table_styles([{
                'selector': 'caption',
                'props': [
                    ('font-size', '16px')
                ]
             }])\
            .set_caption('Share of top 10 Provider/Company Name')\
            .set_properties(padding='10px', border='2px solid white')\
            .bar(color=bar_color)

,Provider/Company Name
Google LLC,12%
Microsoft,2%
IXL Learning,2%
PBS,1%
Houghton Mifflin Harcourt,1%
The Wikimedia Foundation,1%
Dictionary.com,1%
The College Board,1%
Teaching.com,1%
McGraw-Hill PreK-12,1%


**What do we observe**

1) Google, hands down, has the largest share of products to offer on the learning platform.

## Top sectors

In [91]:
pd.DataFrame(lp_daily_eng_df['Sector(s)']\
            .value_counts(normalize=True))[-10:]\
            .style\
            .format('{:.0%}')\
            .set_table_styles([{
                'selector': 'caption',
                'props': [
                    ('font-size', '16px')
                ]
             }])\
            .set_caption('Share of Sector(s)')\
            .set_properties(padding='10px', border='2px solid white')\
            .bar(color=bar_color)

,Sector(s)
PreK-12,41%
PreK-12; Higher Ed; Corporate,39%
PreK-12; Higher Ed,19%
Corporate,1%
Higher Ed; Corporate,0%


## Top 10 Primary Essential Function

In [92]:
pd.DataFrame(lp_daily_eng_df['Primary Essential Function']\
            .value_counts(normalize=True))[:10]\
            .style\
            .format('{:.0%}')\
            .set_table_styles([{
                'selector': 'caption',
                'props': [
                    ('font-size', '16px')
                ]
             }])\
            .set_caption('Share of top 10 Primary Essential Function')\
            .set_properties(padding='10px', border='2px solid white')\
            .bar(color=bar_color)

,Primary Essential Function
LC - Digital Learning Platforms,21%
"LC - Sites, Resources & Reference",13%
LC - Content Creation & Curation,11%
LC - Study Tools,9%
LC/CM/SDO - Other,6%
"LC - Sites, Resources & Reference - Games & Simulations",6%
"LC - Sites, Resources & Reference - Digital Collection & Repository",4%
"LC - Sites, Resources & Reference - Streaming Services",4%
LC - Courseware & Textbooks,3%
LC - Study Tools - Q&A,2%


**What do we observe**

1) Learning Curriculum (LC) appears to be most frequently uses essential function on the platform.

## Share in frequency of usage of best and worst products over time

In [93]:
%%time
product_usage_df = data_for_sparkline(lp_daily_eng_df, ['Product Name', 'usage_month'], 'time', max_time='2020-12',
                              min_time='2020-01',
                              is_product=True,
                              is_multi_level=False,
                              cust_sparkline=False)

CPU times: user 1.61 s, sys: 735 ms, total: 2.34 s
Wall time: 2.34 s


<a id=top_10_product_share_frequency></a>

In [94]:
product_usage_df.sort_values(['growth', 'Product Name'], ascending=[False, True], kind='mergesort')[:10].style\
                                            .format('{:.1%}', subset=product_usage_df.columns.drop(['trend']))\
                                            .set_table_styles([{
                                                'selector': 'caption',
                                                'props': [
                                                    ('font-size', '16px')
                                                ]
                                            }])\
                                            .set_caption('Share in frequency of usage of top 10 products')\
                                            .set_properties(padding='10px', border='2px solid white')\
                                            .background_gradient(cmap='RdYlGn', subset=grad_cols, axis=1)\
                                            .background_gradient(cmap='RdYlGn', subset=['growth'], axis=0)
#                                            .apply(highlight_table, args=(mean_prod_state_usage,))

,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,2020-11,2020-12,trend,growth
Product Name,,,,,,,,,,,,,,
Microsoft Office 365,0.0%,0.0%,0.1%,7.8%,13.6%,10.5%,9.5%,10.6%,12.3%,12.6%,11.9%,11.0%,▁▁▁▅█▆▆▆▇█▇▇,72.0%
Pandora,0.0%,0.0%,2.6%,13.6%,12.3%,8.4%,7.0%,8.2%,12.1%,12.7%,11.8%,11.1%,▁▁▂█▇▅▅▅▇█▇▇,61.0%
Jamboard,1.2%,1.5%,2.1%,5.6%,5.7%,1.7%,0.8%,3.7%,8.0%,1.7%,0.0%,68.1%,▁▁▁▂▂▁▁▁▂▁▁█,40.0%
Seesaw : The Learning Journal,0.6%,0.5%,1.9%,4.5%,3.0%,7.2%,8.3%,9.6%,15.7%,16.7%,16.2%,16.0%,▁▁▂▃▂▄▄▅████,32.0%
Didax,1.1%,1.0%,1.8%,4.6%,4.9%,1.9%,0.9%,6.2%,21.1%,20.9%,19.6%,16.1%,▁▁▁▂▂▁▁▃██▇▆,25.0%
GoToMeeting,1.0%,1.7%,8.1%,17.3%,13.5%,8.9%,5.8%,6.3%,8.5%,9.5%,10.2%,9.2%,▁▁▄█▆▄▃▃▄▅▅▅,20.0%
Leveled Literacy Intervention,2.4%,3.1%,2.6%,2.7%,1.8%,0.7%,0.5%,1.9%,15.1%,23.5%,24.3%,21.4%,▂▂▂▂▁▁▁▁▅██▇,20.0%
WebEx,1.1%,1.4%,6.0%,13.1%,11.2%,8.8%,7.3%,7.4%,11.2%,12.2%,11.2%,9.2%,▁▁▄█▇▅▅▅▇▇▇▆,20.0%
Google Drawings,1.5%,3.0%,11.0%,11.0%,10.9%,7.5%,5.8%,6.8%,10.7%,11.2%,10.6%,10.0%,▁▂███▅▄▅███▇,17.0%


**What do we observe**

1) Barring the first quarter, Microsoft Office 365 showed phenomenal growth in the frequency of its usage during 2020; this is followed by Pandora and Jamboard in that order.

2) GoToMeeting and WebEx appear to be choice of tool for online meetings as opposed to Zoom, which is popular with the corporate world.

3) Surprisingly, Google products don't show up here despite occuring in the top 10 frequently used products.

In [95]:
product_usage_df[product_usage_df.index.str.contains('Google')].sort_values(['growth', 'Product Name'], ascending=[False, True], kind='mergesort')[:20].style\
                                            .format('{:.1%}', subset=product_usage_df.columns.drop(['trend']))\
                                            .set_table_styles([{
                                                'selector': 'caption',
                                                'props': [
                                                    ('font-size', '16px')
                                                ]
                                            }])\
                                            .set_caption('Share in frequency of usage of top 20 Google products')\
                                            .set_properties(padding='10px', border='2px solid white')\
                                            .background_gradient(cmap='RdYlGn', subset=grad_cols, axis=1)\
                                            .background_gradient(cmap='RdYlGn', subset=['growth'], axis=0)
#                                            .apply(highlight_table, args=(mean_prod_state_usage,))

,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,2020-11,2020-12,trend,growth
Product Name,,,,,,,,,,,,,,
Google Drawings,1.5%,3.0%,11.0%,11.0%,10.9%,7.5%,5.8%,6.8%,10.7%,11.2%,10.6%,10.0%,▁▂███▅▄▅███▇,17.0%
Pear Deck for Google Slides,7.9%,7.9%,6.1%,5.2%,5.4%,1.7%,1.0%,4.2%,16.0%,17.7%,14.7%,12.3%,▄▄▃▃▃▁▁▂▇█▇▆,4.0%
Google Voice,6.3%,6.2%,9.7%,12.0%,10.3%,5.6%,4.1%,6.3%,9.7%,10.1%,10.4%,9.4%,▃▃▆█▇▂▁▃▆▆▇▆,3.0%
Google Calendar,8.0%,7.8%,9.1%,9.2%,9.2%,7.2%,6.7%,7.2%,9.0%,9.2%,8.9%,8.6%,▅▄███▂▁▃▇█▇▆,1.0%
Google Arts & Culture,9.4%,9.2%,11.1%,10.6%,10.3%,5.0%,2.9%,4.6%,9.3%,9.3%,8.8%,9.7%,▇▆██▇▃▁▂▆▆▆▇,0.0%
Google Books,8.8%,8.6%,9.4%,9.0%,9.1%,6.1%,4.7%,5.6%,9.6%,10.4%,9.7%,9.1%,▆▆▇▆▆▃▁▂▇█▇▆,0.0%
Google Chrome,8.6%,8.4%,10.0%,9.7%,9.2%,6.4%,5.2%,6.4%,9.5%,9.5%,8.9%,8.4%,▆▆██▇▃▁▃▇▇▆▆,-0.0%
Google Classroom,8.2%,8.1%,9.1%,8.8%,9.0%,7.8%,7.6%,8.1%,8.4%,8.5%,8.1%,8.2%,▄▃█▇▇▂▁▃▅▅▃▄,-0.0%
Google Docs,8.2%,8.1%,8.9%,8.6%,8.7%,8.0%,7.9%,8.3%,8.4%,8.5%,8.2%,8.4%,▃▂█▆▇▂▁▄▄▆▃▄,0.0%


**What do we observe**

1) Google Docs, Google Drive, Google Classrooms and Google Forms that appeared to have a higher share in terms of frequency of usage actually have had no change in growth in the last 12 months. However, what is worth noting is that
their monthly usage rate has been mostly uniform unlike Microsoft Office 365 and Pandora which saw a spike in usage  rates only in the last 4 months, which explains the very high growth rate.

In [96]:
product_usage_df.sort_values(['growth', 'Product Name'], ascending=[False, True], kind='mergesort')[-10:].style\
                                            .format('{:.1%}', subset=product_usage_df.columns.drop(['trend']))\
                                            .set_table_styles([{
                                                'selector': 'caption',
                                                'props': [
                                                    ('font-size', '16px')
                                                ]
                                            }])\
                                            .set_caption('Share in frequency of usage of bottom 10 products')\
                                            .set_properties(padding='10px', border='2px solid white')\
                                            .background_gradient(cmap='RdYlGn', subset=grad_cols, axis=1)\
                                            .background_gradient(cmap='RdYlGn', subset=['growth'], axis=0)

,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,2020-11,2020-12,trend,growth
Product Name,,,,,,,,,,,,,,
Google Training Center,6.1%,4.6%,14.3%,22.1%,15.4%,6.9%,4.2%,6.5%,10.5%,4.8%,2.4%,2.1%,▂▂▅█▆▃▂▃▄▂▁▁,-8.0%
Google Play Music,10.7%,11.0%,11.7%,10.8%,9.9%,6.2%,4.8%,6.5%,9.9%,9.5%,5.4%,3.6%,▇▇█▇▆▃▂▃▆▆▃▁,-9.0%
Fluency Tutor,9.0%,8.8%,10.3%,11.6%,9.9%,5.1%,3.3%,5.3%,12.2%,12.2%,10.6%,1.7%,▆▆▇█▆▃▂▃██▇▁,-13.0%
Google Currents,15.9%,15.1%,17.7%,14.9%,11.9%,5.2%,2.4%,1.8%,4.1%,4.7%,3.5%,2.8%,▇▇█▇▅▃▁▁▂▂▂▁,-13.0%
Savvas EasyBridge,26.0%,18.7%,10.6%,8.8%,8.0%,2.9%,1.5%,3.1%,6.6%,5.7%,4.6%,3.5%,█▆▄▃▃▁▁▁▂▂▂▂,-15.0%
Savvas Realize,16.6%,16.2%,19.2%,19.8%,17.8%,7.1%,0.4%,0.7%,0.8%,0.7%,0.4%,0.4%,▇▇██▇▃▁▁▁▁▁▁,-26.0%
PhET Interactive Science Simulations,23.9%,25.3%,22.6%,15.4%,10.3%,1.7%,0.0%,0.2%,0.3%,0.0%,0.0%,0.3%,██▇▅▄▁▁▁▁▁▁▁,-30.0%
Microsoft XNA Game Studio,27.6%,26.3%,31.0%,11.4%,0.7%,0.5%,0.3%,0.6%,0.5%,0.5%,0.3%,0.3%,▇▇█▄▁▁▁▁▁▁▁▁,-32.0%
Google Science Journal,27.7%,27.4%,34.6%,7.9%,0.7%,0.2%,0.2%,0.2%,0.3%,0.3%,0.3%,0.2%,▇▇█▃▁▁▁▁▁▁▁▁,-33.0%


### Share in engagement index of best products

In [97]:
%%time
product_eng_usage_df = data_for_sparkline(lp_daily_eng_df, ['Product Name', 'usage_month'], 'time', max_time='2020-12',
                              min_time='2020-01',
                              agg_var='engagement_index',
                              is_product_eng=True,
                              is_multi_level=False,
                              cust_sparkline=False)

CPU times: user 1.9 s, sys: 672 ms, total: 2.57 s
Wall time: 2.57 s


In [98]:
product_eng_usage_df.sort_values(['growth', 'Product Name'], ascending=[False, True], kind='mergesort')[:10].style\
                                            .format('{:.1%}', subset=product_eng_usage_df.columns.drop(['trend']))\
                                            .set_table_styles([{
                                                'selector': 'caption',
                                                'props': [
                                                    ('font-size', '16px')
                                                ]
                                            }])\
                                            .set_caption('Share in engagement of top 10 products')\
                                            .set_properties(padding='10px', border='2px solid white')\
                                            .background_gradient(cmap='RdYlGn', subset=grad_cols, axis=1)\
                                            .background_gradient(cmap='RdYlGn', subset=['growth'], axis=0)
#                                            .apply(highlight_table, args=(mean_prod_state_usage,))

,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,2020-11,2020-12,trend,growth
Product Name,,,,,,,,,,,,,,
Microsoft Office 365,0.0%,0.0%,0.0%,10.0%,14.6%,6.7%,3.7%,10.4%,16.9%,14.4%,12.7%,10.6%,▁▁▁▅▇▄▃▅█▇▆▅,153.0%
Pandora,0.0%,0.0%,2.9%,18.4%,11.2%,4.1%,2.3%,6.2%,14.7%,15.3%,13.4%,11.6%,▁▁▂█▅▃▂▃▇▇▆▅,150.0%
Seesaw : The Learning Journal,0.0%,0.0%,0.0%,0.0%,0.0%,1.4%,0.5%,5.4%,26.1%,25.3%,20.9%,20.3%,▁▁▁▁▁▁▁▂██▇▆,146.0%
Jamboard,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,99.9%,▁▁▁▁▁▁▁▁▁▁▁█,143.0%
Google Drawings,0.0%,2.5%,12.9%,13.0%,9.9%,2.7%,0.7%,4.9%,15.5%,15.5%,11.7%,10.8%,▁▂▇▇▅▂▁▃██▆▆,93.0%
Zoom,0.0%,0.0%,5.0%,10.9%,7.3%,2.6%,0.6%,7.0%,21.1%,16.2%,13.6%,15.8%,▁▁▃▅▃▂▁▃█▆▅▆,92.0%
Blindside Networks,0.0%,0.0%,11.3%,12.5%,4.0%,0.1%,0.1%,2.5%,15.6%,18.1%,19.3%,16.3%,▁▁▅▆▂▁▁▂▇██▇,90.0%
Wistia,0.3%,0.7%,7.0%,15.2%,4.6%,1.3%,1.0%,1.8%,11.7%,18.2%,20.4%,17.5%,▁▁▃▆▃▁▁▂▅▇█▇,39.0%
ClassDojo for Students,0.5%,0.8%,2.0%,1.6%,0.9%,0.4%,0.7%,15.3%,27.0%,20.6%,16.2%,14.1%,▁▁▁▁▁▁▁▅█▆▅▅,32.0%


**What do we observe**

1) With respect to share in enagement, there is not much difference among the top 4 products except Jamboard losing 3rd place to Seesaw: The Learning Journal. Zoom, relative to WebEx and GoToMeetings has had a higher
engagement growth rate among the students and teachers.

In [99]:
product_eng_usage_df[product_eng_usage_df.index.str.contains('Google')].sort_values(['growth', 'Product Name'], ascending=[False, True], kind='mergesort')[:20].style\
                                            .format('{:.1%}', subset=product_eng_usage_df.columns.drop(['trend']))\
                                            .set_table_styles([{
                                                'selector': 'caption',
                                                'props': [
                                                    ('font-size', '16px')
                                                ]
                                            }])\
                                            .set_caption('Share in engagement of top 20 Google products')\
                                            .set_properties(padding='10px', border='2px solid white')\
                                            .background_gradient(cmap='RdYlGn', subset=grad_cols, axis=1)\
                                            .background_gradient(cmap='RdYlGn', subset=['growth'], axis=0)
#                                            .apply(highlight_table, args=(mean_prod_state_usage,))

,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,2020-11,2020-12,trend,growth
Product Name,,,,,,,,,,,,,,
Google Drawings,0.0%,2.5%,12.9%,13.0%,9.9%,2.7%,0.7%,4.9%,15.5%,15.5%,11.7%,10.8%,▁▂▇▇▅▂▁▃██▆▆,93.0%
Google Sheets,2.5%,3.3%,5.5%,12.4%,9.1%,3.2%,1.3%,4.4%,14.6%,16.4%,13.8%,13.6%,▂▂▃▆▅▂▁▂▇█▇▇,15.0%
Google Docs,3.0%,4.5%,7.8%,15.0%,11.1%,3.3%,0.6%,2.8%,13.2%,15.4%,12.3%,11.1%,▂▃▄█▆▂▁▂▇█▇▆,12.0%
Google Arts & Culture,3.0%,6.6%,22.4%,14.5%,15.9%,5.6%,0.6%,1.2%,9.5%,5.8%,5.8%,9.0%,▂▃█▅▆▃▁▁▄▃▃▄,10.0%
Google Calendar,3.5%,4.3%,7.6%,11.3%,8.9%,3.7%,1.2%,5.0%,16.8%,15.9%,11.8%,9.9%,▂▂▄▆▄▂▁▃██▆▅,9.0%
Google Forms,3.7%,5.5%,11.9%,15.9%,12.5%,3.5%,0.2%,3.6%,12.7%,12.1%,9.4%,8.9%,▃▃▆█▆▂▁▂▇▆▅▅,8.0%
Google Classroom,3.9%,5.8%,13.6%,14.3%,11.1%,3.4%,0.2%,3.2%,12.8%,12.5%,9.8%,9.3%,▃▄██▆▃▁▂▇▇▆▆,7.0%
Google Translate,8.7%,12.6%,13.5%,8.8%,7.3%,2.8%,0.8%,2.8%,6.8%,8.3%,12.0%,15.7%,▅▇▇▅▄▂▁▂▄▅▆█,5.0%
Google Training Center,0.5%,1.0%,18.4%,42.5%,20.9%,6.7%,2.7%,4.4%,1.4%,0.4%,0.4%,0.8%,▁▁▄█▄▂▁▂▁▁▁▁,4.0%


**What do we observe**

1) With respect to the mean monthly `engagement_index` for Google products. Some of the products such as Google Sheets, Google Docs and Google Forms have seen a postive growth rate. However, this hasn't been
uniform throughout the year.

### Share in frequency of usage and engagement by State and Product

In [100]:
%%time
product_usage_state_wise_time_df = data_for_sparkline(lp_daily_eng_df, ['state', 'Product Name', 'usage_month'], 'time', max_time='2020-12',
                              min_time='2020-01',
                              is_product_combo=True,
                              is_multi_level=False,
                              cust_sparkline=False)

CPU times: user 7.56 s, sys: 1.09 s, total: 8.65 s
Wall time: 8.44 s


<a id=top10_freq_prod_usage></a>

In [101]:
product_usage_state_wise_time_df.sort_values(['growth', 'state'], ascending=[False, True], kind='mergesort')[:10].style\
                                            .format('{:.1%}', subset=product_usage_state_wise_time_df.columns.drop(['trend']))\
                                            .set_table_styles([{
                                                'selector': 'caption',
                                                'props': [
                                                    ('font-size', '16px')
                                                ]
                                            }])\
                                            .set_caption('Share in frequency of usage of top 10 States and their products used')\
                                            .set_properties(padding='10px', border='2px solid white')\
                                            .background_gradient(cmap='RdYlGn', subset=grad_cols, axis=1)\
                                            .background_gradient(cmap='RdYlGn', subset=['growth'], axis=0)
#                                            .apply(highlight_table, args=(mean_prod_dist_usage,))

**What do we observe**

1) The frequency of usage patterns are very diverse across the top 10 States.

2) Products such as Microsoft Office 365, Jamboard and Pandora were used heavily in States such as Connecticut, Illinois and Utah with a high growth rate. These products
started seeing usage as early as April, and they continued with double digit shares for the rest of that year.

3) Other States such as New York, Connecticut and Indiana, especially with respect to their usage of Jamboard reveal that it experienced its most use in December.

In [102]:
%%time
product_usage_eng_state_wise_time_df = data_for_sparkline(lp_daily_eng_df, ['state', 'Product Name', 'usage_month'], 'time', max_time='2020-12',
                              min_time='2020-01',
                              agg_var='engagement_index',
                              is_product_combo_eng=True,
                              is_multi_level=False,
                              cust_sparkline=False)

CPU times: user 13.7 s, sys: 923 ms, total: 14.7 s
Wall time: 14.6 s


<a id=share_eng_top_10_states_and_prod></a>

In [103]:
product_usage_eng_state_wise_time_df.sort_values(['growth', 'state'], ascending=[False, True], kind='mergesort')[:10].style\
                                            .format('{:.1%}', subset=product_usage_eng_state_wise_time_df.columns.drop(['trend']))\
                                            .set_table_styles([{
                                                'selector': 'caption',
                                                'props': [
                                                    ('font-size', '16px')
                                                ]
                                            }])\
                                            .set_caption('Share in engagement of top 10 States and their products used')\
                                            .set_properties(padding='10px', border='2px solid white')\
                                            .background_gradient(cmap='RdYlGn', subset=grad_cols, axis=1)\
                                            .background_gradient(cmap='RdYlGn', subset=['growth'], axis=0)
#                                            .apply(highlight_table, args=(mean_prod_dist_usage,))

,,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,2020-11,2020-12,trend,growth
state,Product Name,,,,,,,,,,,,,,
Texas,Schoology,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,10.9%,29.5%,26.6%,17.2%,15.7%,▁▁▁▁▁▁▁▄█▇▅▅,266.0%
New York,Jamboard,0.0%,0.0%,0.0%,0.1%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,99.9%,▁▁▁▁▁▁▁▁▁▁▁█,186.0%
California,Seesaw : The Learning Journal,0.0%,0.0%,0.0%,0.0%,0.0%,1.0%,0.5%,9.6%,29.0%,26.7%,17.4%,15.8%,▁▁▁▁▁▁▁▃█▇▅▅,179.0%
New Jersey,Seesaw : The Learning Journal,0.0%,0.0%,0.0%,0.0%,0.0%,5.3%,0.1%,0.0%,12.8%,32.7%,19.1%,29.9%,▁▁▁▁▁▂▁▁▄█▅▇,176.0%
Massachusetts,Blindside Networks,0.0%,0.0%,0.2%,0.2%,0.1%,0.0%,0.1%,0.0%,18.8%,26.9%,28.9%,24.7%,▁▁▁▁▁▁▁▁▆▇█▇,173.0%
North Carolina,Blindside Networks,0.0%,0.0%,1.7%,1.7%,0.7%,0.1%,0.0%,4.9%,18.0%,29.5%,24.8%,18.5%,▁▁▁▁▁▁▁▂▅█▇▅,172.0%
Washington,Seesaw : The Learning Journal,0.0%,0.0%,0.0%,0.0%,0.0%,3.7%,0.3%,0.3%,26.6%,23.3%,26.0%,19.8%,▁▁▁▁▁▂▁▁█▇█▆,170.0%
New York,Seesaw : The Learning Journal,0.0%,0.0%,0.0%,0.0%,0.0%,1.4%,0.5%,0.1%,15.8%,24.2%,31.5%,26.3%,▁▁▁▁▁▁▁▁▅▆█▇,168.0%
Indiana,Seesaw : The Learning Journal,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,0.0%,19.1%,25.6%,17.2%,23.6%,14.5%,▁▁▁▁▁▁▁▆█▆▇▅,164.0%


**What do we observe**

1) A contrasting picture emerges when we look at share in engagement of top 10 States and their products used. Some of the States such as Connecticut, Illinois, Utah, Indiana and North Carolina don't appear at all.

2) Jamboard appears to have found traction in one or more NY districts only in December.

3) Seasaw: The Learning Journal seemed to be a popular learning tool, across several States, during the last quarter in 2020.

4) Microsoft Office 365 doesn't feature in the top 10 across any of the States, implying that there is little correlation between frequency of using a given product and its engagement.

In [104]:
google_products = ['Google Docs', 'Google Sheets', 'Google Forms', 'Google Drive', 'Google Classroom']
product_usage_eng_state_wise_time_df.query('`Product Name` == @google_products').sort_values(['growth', 'Product Name'], ascending=[False, True])[:10].style\
                                            .format('{:.1%}', subset=product_usage_eng_state_wise_time_df.columns.drop(['trend']))\
                                            .set_table_styles([{
                                                'selector': 'caption',
                                                'props': [
                                                    ('font-size', '16px')
                                                ]
                                            }])\
                                            .set_caption('Share in engagement of top 5 Google products by States')\
                                            .set_properties(padding='10px', border='2px solid white')\
                                            .background_gradient(cmap='RdYlGn', subset=grad_cols, axis=1)\
                                            .background_gradient(cmap='RdYlGn', subset=['growth'], axis=0)
#                                            .apply(highlight_table, args=(mean_prod_dist_usage,))

**What do we observe**

1) New York (NY) shows engagement with Google Sheets and Google Docs for most part of the year except for sometime in January and during months of June and July. A similar pattern is noted for Wisoncnin with Google Classroom.

2) For States such as Michigan, Tennessee, their usage of the products occurred mostly during the last 4 months of the year.

In [105]:
# add this line to the State health card
most_preferred_product_statewise = product_usage_eng_state_wise_time_df.reset_index().groupby('state').apply(lambda x: x.nlargest(1, ['growth']))[['state', 'Product Name', 'growth']]
most_preferred_product_statewise = most_preferred_product_statewise.rename(columns={'state': 'dummy'}).reset_index(drop=True).rename(columns={'dummy': 'state'})

## Share in engagement of top 10 Primary Essential functions

In [106]:
%%time
primary_func_usage_df = data_for_sparkline(lp_daily_eng_df, ['Primary Essential Function', 'usage_month'], 'time', max_time='2020-12',
                              min_time='2020-01',
                              agg_var='engagement_index',
                              is_product_eng=True,
                              is_multi_level=False,
                              cust_sparkline=False)

CPU times: user 1.43 s, sys: 625 ms, total: 2.05 s
Wall time: 2.05 s


In [107]:
primary_func_usage_df.sort_values(['growth', 'Primary Essential Function'], ascending=[False, True], kind='mergesort')[:10].style\
                                            .format('{:.1%}', subset=primary_func_usage_df.columns.drop(['trend']))\
                                            .set_table_styles([{
                                                'selector': 'caption',
                                                'props': [
                                                    ('font-size', '16px')
                                                ]
                                            }])\
                                            .set_caption('Share in engagement of top 10 essential functions')\
                                            .set_properties(padding='10px', border='2px solid white')\
                                            .background_gradient(cmap='RdYlGn', subset=grad_cols, axis=1)\
                                            .background_gradient(cmap='RdYlGn', subset=['growth'], axis=0)
#                                            .apply(highlight_table, args=(mean_prod_state_usage,))

,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,2020-11,2020-12,trend,growth
Primary Essential Function,,,,,,,,,,,,,,
CM - Virtual Classroom - Video Conferencing & Screen Sharing,0.0%,0.0%,1.7%,7.8%,7.4%,3.0%,0.6%,5.1%,19.6%,18.5%,16.7%,19.6%,▁▁▂▄▄▂▁▃██▇█,121.0%
"LC - Sites, Resources & Reference - Streaming Services",0.4%,0.6%,1.5%,1.5%,1.1%,3.4%,6.4%,8.5%,16.9%,20.5%,19.7%,19.5%,▁▁▁▁▁▂▃▄▇███,38.0%
CM - Classroom Engagement & Instruction - Classroom Management,2.2%,4.8%,14.0%,14.8%,9.4%,2.6%,1.2%,4.7%,13.1%,13.4%,10.3%,9.6%,▂▃██▅▂▁▃▇▇▆▅,13.0%
CM - Teacher Resources - Lesson Planning,4.2%,6.7%,8.3%,6.1%,5.7%,2.1%,1.4%,4.3%,15.0%,13.6%,14.9%,17.8%,▂▃▄▃▃▁▁▂▇▆▇█,13.0%
LC - Content Creation & Curation,3.0%,4.6%,8.0%,14.7%,10.9%,3.2%,0.6%,2.9%,13.2%,15.3%,12.2%,11.3%,▂▃▅█▆▂▁▂▇█▇▆,12.0%
CM - Teacher Resources - Professional Learning,2.6%,3.7%,7.0%,25.3%,11.0%,3.7%,1.5%,3.0%,10.9%,14.2%,9.5%,7.6%,▁▂▃█▄▂▁▁▄▅▃▃,9.0%
LC - Online Course Providers & Technical Skills Development,4.1%,6.5%,10.3%,9.9%,5.4%,1.0%,0.4%,6.0%,17.6%,15.5%,12.4%,10.9%,▂▃▅▅▃▁▁▃█▇▆▅,9.0%
CM - Classroom Engagement & Instruction - Assessment & Classroom Response,3.7%,5.5%,11.9%,15.9%,12.4%,3.5%,0.2%,3.6%,12.7%,12.3%,9.4%,8.9%,▃▃▆█▆▂▁▂▇▆▅▅,8.0%
SDO - Learning Management Systems (LMS),4.0%,5.9%,13.0%,13.9%,10.6%,3.2%,0.2%,3.3%,13.2%,12.8%,10.2%,9.7%,▃▄██▆▃▁▃█▇▆▆,8.0%


**What do we observe**

1) Content Management products - Video conferencing and Screen sharing has seen the most growth off all primary essential functions. Schools started using it right after the schools shutdown towards the end of March. There was some
engagement in April and May, but there was disruption during June and July. Subsequently, its growth picked up nicely in the last 4 months of that year.

## Share in engagement by State and Primary Essential functions over time

In [108]:
%%time
primary_func_eng_usage_df = data_for_sparkline(lp_daily_eng_df, ['state', 'Primary Essential Function', 'usage_month'], 'time', max_time='2020-12',
                              min_time='2020-01',
                              agg_var='engagement_index',
                              is_product_combo_eng=True,
                              is_multi_level=False,
                              cust_sparkline=False)

CPU times: user 3.11 s, sys: 706 ms, total: 3.82 s
Wall time: 3.82 s


<a id=CM_top10_by_state></a>

In [109]:
primary_func_eng_usage_df.sort_values(['growth', 'Primary Essential Function'], ascending=[False, True], kind='mergesort')[:10].style\
                                            .format('{:.1%}', subset=primary_func_eng_usage_df.columns.drop(['trend']))\
                                            .set_table_styles([{
                                                'selector': 'caption',
                                                'props': [
                                                    ('font-size', '16px')
                                                ]
                                            }])\
                                            .set_caption('Share in engagement of top 10 States and their preferred primary essential function')\
                                            .set_properties(padding='10px', border='2px solid white')\
                                            .background_gradient(cmap='RdYlGn', subset=grad_cols, axis=1)\
                                            .background_gradient(cmap='RdYlGn', subset=['growth'], axis=0)
#                                            .apply(highlight_table, args=(mean_prod_state_usage,))

,,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,2020-11,2020-12,trend,growth
state,Primary Essential Function,,,,,,,,,,,,,,
Michigan,CM - Virtual Classroom - Video Conferencing & Screen Sharing,0.0%,0.0%,0.0%,0.4%,0.6%,0.1%,0.0%,0.7%,28.5%,27.2%,23.2%,19.3%,▁▁▁▁▁▁▁▁██▇▆,184.0%
Illinois,CM - Virtual Classroom - Video Conferencing & Screen Sharing,0.0%,0.0%,0.5%,3.8%,2.9%,0.7%,0.3%,11.3%,24.6%,20.1%,16.2%,19.5%,▁▁▁▂▂▁▁▄█▇▆▇,162.0%
Wisconsin,CM - Virtual Classroom - Video Conferencing & Screen Sharing,0.0%,0.0%,2.3%,14.4%,13.0%,5.0%,2.3%,0.5%,26.6%,12.9%,10.6%,12.3%,▁▁▂▅▄▂▂▁█▄▄▄,155.0%
Connecticut,CM - Virtual Classroom - Video Conferencing & Screen Sharing,0.0%,0.0%,1.5%,6.9%,8.1%,3.7%,0.8%,0.5%,17.0%,17.1%,20.0%,24.4%,▁▁▁▃▃▂▁▁▆▆▇█,153.0%
New York,CM - Virtual Classroom - Video Conferencing & Screen Sharing,0.0%,0.0%,3.1%,12.7%,12.6%,7.2%,1.0%,1.8%,14.0%,15.7%,14.6%,17.3%,▁▁▂▆▆▄▁▂▇▇▇█,150.0%
Florida,CM - Virtual Classroom - Video Conferencing & Screen Sharing,0.0%,0.0%,0.1%,4.2%,2.4%,0.4%,0.2%,15.9%,23.5%,25.4%,14.2%,13.7%,▁▁▁▂▂▁▁▅▇█▅▅,134.0%
Indiana,CM - Virtual Classroom - Video Conferencing & Screen Sharing,0.0%,0.0%,0.8%,7.7%,5.9%,1.1%,0.2%,12.5%,15.1%,11.4%,23.1%,22.3%,▁▁▁▃▃▁▁▅▆▄██,134.0%
New Jersey,CM - Virtual Classroom - Video Conferencing & Screen Sharing,0.0%,0.0%,3.4%,20.1%,18.6%,11.2%,1.2%,0.8%,10.5%,12.1%,8.6%,13.5%,▁▁▂█▇▅▁▁▅▅▄▆,134.0%
California,CM - Virtual Classroom - Video Conferencing & Screen Sharing,0.0%,0.0%,0.6%,4.7%,5.1%,1.2%,0.4%,14.0%,23.3%,21.1%,15.2%,14.6%,▁▁▁▂▃▁▁▅█▇▆▅,126.0%


**What do we observe**

1) This makes sense given that most teachers in their schools would have spent time conducting live classrooms.

2) Some States such as Michigan, Illinois, Florida, California and Massachusetts saw relatively lower usage right after school closures. Most of their engagement happened during the last 4 months in that year. This is unlike that observed in States such as Wisconsin, Connecticut, New York, Indiana, and New Jersey that showed relatively better engagement right after school closure.

In [110]:
primary_func_eng_usage_df.query('`Primary Essential Function` == "LC - Sites, Resources & Reference - Streaming Services"')\
                                            .sort_values(['growth', 'Primary Essential Function'], ascending=[False, True], kind='mergesort')[:10].style\
                                            .format('{:.1%}', subset=primary_func_eng_usage_df.columns.drop(['trend']))\
                                            .set_table_styles([{
                                                'selector': 'caption',
                                                'props': [
                                                    ('font-size', '16px')
                                                ]
                                            }])\
                                            .set_caption('Share in engagement of top 10 States and their use of LC - Sites, Resources & References - Streaming Services')\
                                            .set_properties(padding='10px', border='2px solid white')\
                                            .background_gradient(cmap='RdYlGn', subset=grad_cols, axis=1)\
                                            .background_gradient(cmap='RdYlGn', subset=['growth'], axis=0)
#                                            .apply(highlight_table, args=(mean_prod_state_usage,))

,,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,2020-11,2020-12,trend,growth
state,Primary Essential Function,,,,,,,,,,,,,,
Texas,"LC - Sites, Resources & Reference - Streaming Services",0.0%,0.0%,0.0%,0.0%,0.3%,2.1%,4.9%,8.2%,21.2%,22.8%,19.8%,20.6%,▁▁▁▁▁▂▃▄██▇▇,113.0%
New York,"LC - Sites, Resources & Reference - Streaming Services",0.0%,0.4%,1.7%,1.8%,1.5%,5.4%,10.6%,9.3%,16.9%,18.8%,19.3%,14.1%,▁▁▂▂▂▃▅▄▇██▆,63.0%
Tennessee,"LC - Sites, Resources & Reference - Streaming Services",0.1%,0.2%,0.1%,0.0%,0.0%,0.2%,0.3%,7.0%,23.4%,21.6%,22.7%,24.4%,▁▁▁▁▁▁▁▃█▇██,60.0%
Florida,"LC - Sites, Resources & Reference - Streaming Services",0.1%,0.0%,0.0%,0.1%,0.1%,4.7%,10.7%,12.0%,16.6%,20.4%,17.3%,17.9%,▁▁▁▁▁▃▅▅▇█▇▇,53.0%
North Carolina,"LC - Sites, Resources & Reference - Streaming Services",0.2%,0.3%,0.5%,0.3%,0.1%,0.1%,0.4%,3.6%,12.6%,25.8%,23.9%,32.0%,▁▁▁▁▁▁▁▂▄▇▆█,51.0%
Washington,"LC - Sites, Resources & Reference - Streaming Services",0.2%,0.3%,0.6%,0.7%,0.5%,4.1%,7.5%,7.5%,14.4%,16.6%,24.7%,23.1%,▁▁▁▁▁▂▃▃▅▆██,50.0%
California,"LC - Sites, Resources & Reference - Streaming Services",0.2%,0.2%,0.5%,0.6%,0.5%,2.7%,5.2%,10.6%,23.4%,21.7%,17.6%,16.8%,▁▁▁▁▁▂▃▄█▇▆▆,48.0%
Indiana,"LC - Sites, Resources & Reference - Streaming Services",0.2%,0.4%,0.9%,0.6%,0.4%,1.9%,3.6%,12.0%,16.7%,20.7%,22.1%,20.5%,▁▁▁▁▁▂▂▅▆██▇,47.0%
Virginia,"LC - Sites, Resources & Reference - Streaming Services",0.2%,0.4%,0.8%,0.7%,0.7%,3.1%,6.0%,7.0%,17.1%,23.3%,20.9%,19.8%,▁▁▁▁▁▂▃▃▆█▇▇,47.0%


**What do we observe**

1) Slicing the data to look at only "LC - Sites, Resources & Reference - Streaming Services", the second most popular function in terms of engagement growth rate, what we see is that all the States have engaged with this essential function
mostly during the last 4 months of that year.

<a id=top10_SDO></a>

In [111]:
primary_func_eng_usage_df.query('`Primary Essential Function` == "SDO - Learning Management Systems (LMS)"')\
                                            .sort_values(['growth', 'Primary Essential Function'], ascending=[False, True], kind='mergesort')[:10].style\
                                            .format('{:.1%}', subset=primary_func_eng_usage_df.columns.drop(['trend']))\
                                            .set_table_styles([{
                                                'selector': 'caption',
                                                'props': [
                                                    ('font-size', '16px')
                                                ]
                                            }])\
                                            .set_caption('Share in engagement of top 10 States and their use of SDO - Learning Management Systems (LMS)')\
                                            .set_properties(padding='10px', border='2px solid white')\
                                            .background_gradient(cmap='RdYlGn', subset=grad_cols, axis=1)\
                                            .background_gradient(cmap='RdYlGn', subset=['growth'], axis=0)
#                                            .apply(highlight_table, args=(mean_prod_state_usage,))

,,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,2020-11,2020-12,trend,growth
state,Primary Essential Function,,,,,,,,,,,,,,
Wisconsin,SDO - Learning Management Systems (LMS),0.1%,2.1%,9.3%,20.4%,14.7%,1.4%,0.8%,0.1%,14.9%,13.8%,11.9%,10.5%,▁▂▄█▆▁▁▁▆▆▅▅,46.0%
Texas,SDO - Learning Management Systems (LMS),1.0%,1.7%,0.2%,0.0%,5.1%,1.1%,0.5%,10.0%,26.7%,24.1%,15.6%,14.2%,▁▁▁▁▂▁▁▄█▇▅▅,24.0%
District Of Columbia,SDO - Learning Management Systems (LMS),1.7%,2.3%,9.3%,10.9%,9.1%,1.8%,0.8%,3.7%,21.4%,17.6%,11.1%,10.2%,▁▂▄▄▄▁▁▂█▇▄▄,16.0%
Michigan,SDO - Learning Management Systems (LMS),2.8%,4.1%,2.5%,1.3%,2.0%,0.3%,0.0%,1.2%,27.2%,24.8%,18.9%,14.9%,▂▂▂▁▂▁▁▁█▇▆▅,15.0%
New York,SDO - Learning Management Systems (LMS),1.6%,4.3%,19.3%,15.8%,13.0%,6.9%,0.3%,1.0%,11.0%,10.7%,8.3%,8.0%,▁▂█▇▆▃▁▁▅▅▄▄,14.0%
California,SDO - Learning Management Systems (LMS),2.6%,3.7%,8.4%,10.2%,9.5%,1.5%,0.2%,10.1%,19.3%,15.6%,9.9%,9.0%,▂▂▄▅▄▁▁▅█▇▅▄,11.0%
Connecticut,SDO - Learning Management Systems (LMS),3.3%,4.5%,12.4%,14.0%,12.1%,4.8%,0.2%,0.4%,13.0%,13.2%,11.0%,11.1%,▃▃▇█▇▃▁▁██▆▇,11.0%
Illinois,SDO - Learning Management Systems (LMS),3.3%,5.5%,11.7%,12.8%,7.5%,0.8%,0.1%,8.1%,15.7%,13.8%,10.3%,10.3%,▂▃▆▇▄▁▁▅█▇▆▆,10.0%
Tennessee,SDO - Learning Management Systems (LMS),6.3%,10.6%,2.0%,1.2%,0.3%,0.0%,0.0%,9.1%,25.2%,15.6%,12.7%,17.0%,▃▄▂▁▁▁▁▄█▅▅▆,9.0%


**What do we observe**

1) Looking at the School and Districts operations - Learning Management Systems (LMS), which is in the top 10 essential functions based on engagement growth, we can see that States such as Wisconsin, District Of Columbia, New York, Connecticut, and to a lesser extent California and Illiniois had some interactions during the latter half of the first quarter and second quarter, whereas, States such as Texas, Michigan and Tennesse saw this interactions happen mostly in the last 4 months of that year.

## State report card

In [112]:
def prepare_report_card(df, merge_df, states_agg_df, best_prod_df):
    """
    Prepare a summary report card
    showing how each State performed
    # create a report card for the states
    # calculate the count of districts by state
    # calculate the count of healthy districts
    # calculate proportion of healthy districts
    # merge dataframe to capture monthly engagement growth
    # merge dataframe to capture district features
    """

    tmp_report_card = df.loc[:, 'growth'].reset_index().copy()
    tmp_report_card['total_healthy'] = tmp_report_card['growth'].apply(lambda x: 1 if x > 0 else 0)
    state_report_card = pd.DataFrame(tmp_report_card.groupby('state').size()).merge(pd.DataFrame(tmp_report_card.groupby('state')['total_healthy'].sum()), left_index=True, 
                                                          right_index=True).reset_index().rename(columns={0: 'total_districts'})
    state_report_card['percent_healthy'] = state_report_card['total_healthy'] / state_report_card['total_districts']
    state_report_card = state_report_card.merge(merge_df.loc[:, 'growth'], left_on='state', right_index=True)
    
    dist_cols = ['state', 'pct_mean_black_hispanic', 'pct_mean_free', 'county_connections_mean_ratio', 'median_pp_total_raw']

    state_report_card = state_report_card.merge(states_agg_df[dist_cols], left_on='state', right_on='state')
    state_report_card = state_report_card.reindex(columns=dist_cols + ['total_districts', 'total_healthy', 'percent_healthy', 'growth'])
    state_report_card.rename(columns={'growth': 'monthly_engagement_growth_rate'}, inplace=True)
    
#     state_report_card = state_report_card.merge(prod_df.loc[:, 'growth'], left_on='state', right_index=True)
#     state_report_card = state_report_card.rename(columns={'growth': 'monthly_product_usage_growth_rate'})
    state_report_card = state_report_card.merge(best_prod_df, left_on='state', right_on='state')
    state_report_card.rename(columns={'Product Name': 'preferred_product', 
                                     'growth': 'monthly_product_engagement_rate'}, inplace=True)
    state_report_card = state_report_card.reindex(columns=dist_cols + ['total_districts', 'total_healthy', 'percent_healthy', 
                                                                       'preferred_product',
                                                                       'monthly_product_engagement_rate',
                                                                       'monthly_engagement_growth_rate'])
    return state_report_card

<a id=state_report> </a>

In [113]:
state_report_card = prepare_report_card(district_wise_mom_engagement_growth,
                                       state_wise_mom_engagement_growth,
                                       states_agg,
                                       most_preferred_product_statewise)
state_report_card\
            .sort_values('monthly_engagement_growth_rate', ascending=False)\
            .set_index('state')\
            .style\
            .format('{:.1%}', subset=['pct_mean_black_hispanic', 'pct_mean_free', 
                                      'percent_healthy', 'monthly_engagement_growth_rate',
                                     'monthly_product_engagement_rate'])\
            .format('{:.0f}', subset=['median_pp_total_raw'])\
            .set_table_styles([{
                                'selector': 'caption',
                                'props': [
                                    ('font-size', '16px')
                                ]
                            }])\
            .set_caption('State health report card, Jan 2020 - Dec 2020')\
            .set_properties(padding='10px', border='2px solid white')\
            .bar(align='mid', color=bar_color)

,pct_mean_black_hispanic,pct_mean_free,county_connections_mean_ratio,median_pp_total_raw,total_districts,total_healthy,percent_healthy,preferred_product,monthly_product_engagement_rate,monthly_engagement_growth_rate
state,,,,,,,,,,
District Of Columbia,76.7%,nan%,0.590000,19000,3,2,66.7%,Seesaw : The Learning Journal,147.0%,15.0%
New York,20.0%,42.5%,0.590000,20714,8,5,62.5%,Jamboard,186.0%,14.0%
Texas,60.0%,50.0%,0.590000,9000,2,0,0.0%,Schoology,266.0%,12.0%
Michigan,50.0%,60.0%,0.590000,12000,2,2,100.0%,Zoom,96.0%,11.0%
Illinois,31.1%,35.6%,0.590000,13556,18,14,77.8%,Jamboard,161.0%,10.0%
Connecticut,21.3%,26.7%,0.590000,nan,30,29,96.7%,Seesaw : The Learning Journal,154.0%,9.0%
California,41.7%,33.3%,0.590000,nan,12,7,58.3%,Seesaw : The Learning Journal,179.0%,9.0%
Florida,50.0%,50.0%,0.590000,7000,1,1,100.0%,Jamboard,162.0%,8.0%
Virginia,25.0%,50.0%,0.590000,11500,4,3,75.0%,Seesaw : The Learning Journal,139.0%,8.0%


**How is the data prepared**

1) This dataset summarizes the following attributes:

    a) district features

    b) the total number of districts

    c) total healthy districts, which is based on whether that district experienced a positive engagement rate or not in 2020

    d) the most popular product used based on engagement growth

    e) mean monthly product eneagement rate, and

    f) mean monthly engagement growth rate

All of these attributes are collected at the State level and joined together; these are sorted in descending order based on mean monthly engagement growth rate.

**What do we observe**

1) New York and District Of Columbia are comparable in terms of similar median expendiure per pupil. However, District Of Columbia has a relatively better district health, and a comparable monthly engagement growth rate compared with New York.

2) In terms of having a high percentage of Black/Hispanic students, the top 2 States are:a) District Of Columbia and b) Arizona at 77% and 90%, each, respectively.

3) With respect to offering free/reduced price meals, the highest percentage of students come from the following States: Michigan, Florida, Indiana, and Minnesota. Of these, Michigan has a relatively better monthly engagement growth rate. However, all of these States have done a reasonable job in ensuring their districts showed reasonably healthy engagement.

4) Connecticut and Utah have the highest number of districts. However, Connecticut has done a better job in growing its monthly engagement rate at 9% relative to Utah's 7% during the same period.

5) SeeSaw: The Learning Journal is the most popular product across most States based on `engagement_index`.

## Summary

The pandemic threw everyones life out of gear for most part of last year, including the current year. While some countries are still in lockdown, others are emerging cautiously out of it while ensuring its population are vaccinated. In the US,
only 27 states were closed according to [Edweek](https://www.edweek.org/leadership/the-coronavirus-spring-the-historic-closing-of-u-s-schools-a-timeline/2020/07). It was not until March 25 that all US publicschools were ordered to shutdown; this
impacted the morale of both students and teachers, and it took another month before online learning became almost the defacto way of learning going forward.

1) The [mean monthly engagement index](#mean_monthly_engagement_index) shows that news of the closure lowered the mean monthly `enagement_index`, which was at its peak in April, until it reached its lowest in July. However, we notice a reveral right after July; the mean `engagement_index` peaks in October before declining slightly for the rest of 2020.

2) The scars of COVID-19 pandemic are here to stay. Governments, Corporations, and the public in general will continue to practice COVID appropriate behaviour for years to come. In that light, we have seen that some US States [engagement growth](#State-wise_monthly_share_of_engagement_index) such as New York, Connecticut, etc. were able to cope relatively better than others such as Ohio, Wisconsin and Minnesota during the pandemic. Yet others such as District of Columbia, Texas saw positive engagement rates during the last 4 months in 2020. This tells us that some States were better prepared. Even if things return to normal, schools will be better prepared to switch to online learning with minimal disruption; this will be supported by much improved technology in the future.

3) The experience with respect to usage of various digital technology is quite varied during 2020. Just based on frequency of product usage, this is what we see [frequency of usage](#top_10_product_share_frequency). When we include the State dimension, and make a comparison between [share in frequency of usage by State and Product](#top10_freq) and [engagement share by State and Product](#share_eng_top_10_states_and_prod), a different picture emerges - one which reveals a weak correlation between frequency of using a given product and its engagement. The tables show that usage pattern varies significantly. Some States show a lot of activity with certain products during the last 4 months, while others have shown activity during April and May, including during the last 4 months. Yet others have only shown activity during the first quarter, and none later.

4) There is a strong positive correlation between `pct_access` and `engagement_index` and between `pct_mean_free` and `pct_mean_black_hispanic` respectively [correlation between district features](#correlation_matrix); this is reinforced by these [parallel coordinate charts](#parallel_coord). By geography, the most preferred essential function, [share of top 10 essential functions by State](#CM_top10_by_state) regardless of ethnicity, is CM - Virtual Classroom.

5) By looking at the table [share of engagement for top 10 States and their use of SDO-LMS](#top10_SDO), there is some indication to show that early intervention by the school districts helps with improving the engagement rates [State health report](#state_report); Wisconsin might be an outlier. As shown in the table the intervention in the month of Jan was very low, however, the last 4 months saw some activity which boosted its SDO growth but this wasn't enough to improve the overall engagement growth rate.


## Potential solutions

1) Weak performing districts can learn from their stronger counterparts in that State to find out what they did differently that helped them navigate the pandemic relatively better.

2) Teachers should be financially incentivised to upskill.

3) Teachers should divide the classroom into teams, and organize online quizzes, brainstorming sessions using various online tools, etc. Teachers should leverage the help of highly motivated students
to help support the slow learners. Such students should be rewarded via the school's grading system.

4) Local companies should support such schools by orgnanising online hackathons with attractive prizes to encourage student participation.

## References

[EdWeek](https://www.edweek.org/leadership/the-coronavirus-spring-the-historic-closing-of-u-s-schools-a-timeline/2020/07)

[Correlation Plot](https://community.plotly.com/t/correlation-plot-with-mask/29185)

[Subplots](https://plotly.com/python/subplots/)

[Bar Color Change](https://community.plotly.com/t/plotly-express-bar-colour-change/36210)

[Pandas Sort](https://realpython.com/pandas-sort-python/)

<a id=correlation_matrix></a>